<a href="https://colab.research.google.com/github/gseetha04/stakeholder-classification/blob/main/error_analysis_academicdata.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from pandas import read_csv
import pandas as pd
import math
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
import torch

In [ ]:
# Method to get level_1 labels from level_2 labels
max_length=0
def label_types (row):
  global max_length
  length = len(row['Label'])
  max_length = max(max_length,length)
  if length == 4 :
    return row['Label'][0:3]
  else: return row['Label']

In [ ]:
training_data = read_csv('organizational-performance-dataset.csv')
training_data['Label'] = pd.Categorical(training_data['Label'])
training_data['Label_num_level_2'] = training_data['Label'].cat.codes
training_data['Label_num_level_2'].astype('int64')
training_data['Label_level_1'] = training_data.apply (lambda row: label_types(row), axis=1)
training_data['Label_level_1'] = pd.Categorical(training_data['Label_level_1'])
training_data['Label_num_level_1'] = training_data['Label_level_1'].cat.codes
training_data['Label_num_level_1'].astype('int64')
training_data.head(20)

,Concept_ID,Concept,Label,Hypothesis Sentence,NON-PERFORMANCE,PERFORMANCE,Label_num_level_2,Label_level_1,Label_num_level_1
0,1,change in customer satisfaction,CUS2,"(' , a change in customer satisfaction will be...",0,1,1,CUS,0
1,2,change in financial performance,INV1,"(' , a change in customer satisfaction will be...",0,1,7,INV,2
2,1,customer satisfaction,CUS2,"(' , customer satisfaction and behavioral inte...",0,1,1,CUS,0
3,2,behavioral intention,NP,"(' , customer satisfaction and behavioral inte...",1,0,11,NP,3
4,1,employee participation,EMP1,"(' , employee participation is positively rela...",0,1,4,EMP,1
5,2,organizational performance,INV3,"(' , employee participation is positively rela...",0,1,9,INV,2
6,1,orientation of iso 14001 practices,SOC3,"(' , it was postulated that ﬁrms going for iso...",0,1,13,SOC,4
7,2,performance,NP,"(' , it was postulated that ﬁrms going for iso...",1,0,11,NP,3
8,1,strategic proactivity capability,NP,"(' , regarding the inﬂuence of a strategic pro...",1,0,11,NP,3
9,2,environmental proactiveness,SOC3,"(' , regarding the inﬂuence of a strategic pro...",0,1,13,SOC,4


In [ ]:
training_data.to_csv('/content/training_data.csv')

In [ ]:
from sklearn.linear_model import LogisticRegression
import sklearn.model_selection
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline, FeatureUnion

0      1
1      1
2      1
3      0
4      1
      ..
684    1
685    1
686    0
687    0
688    1
Name: PERFORMANCE, Length: 689, dtype: int64
0      0
1      2
2      0
3      3
4      1
      ..
684    1
685    2
686    3
687    3
688    4
Name: Label_num_level_1, Length: 689, dtype: int8
0       1
1       7
2       1
3      11
4       4
       ..
684     4
685     7
686    11
687    11
688    13
Name: Label_num_level_2, Length: 689, dtype: int8


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)


In [ ]:
print(train_texts.shape)
print(val_texts.shape)
print(train_labels.shape)
print(val_labels.shape)

(552,)
(137,)
(552,)
(137,)


In [ ]:
k=5
seed = 1
kf = StratifiedKFold(n_splits=k,random_state=seed, shuffle=True)
label_count = {'PERFORMANCE':2,'Label_num_level_1':5,'Label_num_level_2':16}
for label, count in label_count.items():
  x=training_data
  y=training_data[label]
  metrics=[]
  i=1
  for train_index, val_index in kf.split(x,y):
    train_texts, val_texts =x['Concept'].iloc[train_index],x['Concept'].iloc[val_index]
    train_labels, val_labels =y.iloc[train_index],y.iloc[val_index]
    list_text = []
    for words in train_texts:
      list_text.append(words)

    # Preprocessing data to feed into MultinomialNB Classifier
    # Tried ngram_range=[1,1],[2,2],[3,3]
    count_vect = CountVectorizer(ngram_range=[1,3])
    tfidf_transformer=TfidfTransformer()

    # Pipeline using CountVectorizer and TfidfTransformer
    pipeline = Pipeline([('count_vect',count_vect),('tfidf_transformer',tfidf_transformer)])
    pipeline_output = pipeline.fit_transform(list_text)
    param_grid_ = {'C': [1e-5, 1e-3, 1e-1, 1e0, 1e1, 1e2]}
    clf = GridSearchCV(LogisticRegression(solver='lbfgs', max_iter=1000, class_weight='balanced'), cv=5, param_grid=param_grid_).fit(pipeline_output, train_labels)
    # Evaluation Model performance on the Test data
    sentence_list = val_texts.to_list()
    pipeline_test = Pipeline([('count_vect',count_vect),('tfidf_transformer',tfidf_transformer)])
    pipeline_test_output = pipeline_test.transform(sentence_list)
    predicted = clf.predict(pipeline_test_output)

    # #Predicting Causal variables into Performance and Non Performance groups
    # variables_list = merged_causal_statements['Variables'].to_list()
    # pipeline_causal=pipeline_test.transform(variables_list)
    # predicted_causal = clf.predict(pipeline_causal)
    # classification_prediction_causal_list =[]
    # for variable, classification in zip(variables_list, predicted_causal):
    #   classification_prediction_causal_list.append(classification)

    classification_prediction_list =[]
    for sentence, classification in zip(sentence_list, predicted):
      classification_prediction_list.append(classification)
    # Naive_Bayes Model metrics
    classification_list = val_labels.to_list()
    indices = val_index
    training_data.loc[indices, "Logistic_{}_Predictions".format(label)]=classification_prediction_list
    # merged_causal_statements["Logistic_{}_Predictions_{}".format(label,i)]=classification_prediction_causal_list
    i = i+1

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in 

In [ ]:
print("Logistic")
print("PERFORMANCE")
print(classification_report(training_data['PERFORMANCE'], training_data['Logistic_PERFORMANCE_Predictions'], zero_division = 1))
print("Label_num_level_1")
print(classification_report(training_data['Label_num_level_1'], training_data['Logistic_Label_num_level_1_Predictions'], zero_division = 1))
print("Label_num_level_2")
print(classification_report(training_data['Label_num_level_2'], training_data['Logistic_Label_num_level_2_Predictions'], zero_division = 1))

Logistic
PERFORMANCE
              precision    recall  f1-score   support

           0       0.85      0.85      0.85       421
           1       0.77      0.76      0.77       268

    accuracy                           0.82       689
   macro avg       0.81      0.81      0.81       689
weighted avg       0.82      0.82      0.82       689

Label_num_level_1
              precision    recall  f1-score   support

           0       0.65      0.72      0.68        43
           1       0.71      0.73      0.72        55
           2       0.68      0.87      0.77        87
           3       0.91      0.81      0.86       421
           4       0.71      0.83      0.77        83

    accuracy                           0.81       689
   macro avg       0.73      0.79      0.76       689
weighted avg       0.83      0.81      0.81       689

Label_num_level_2
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         4
           1      

In [ ]:
k=5
seed = 2
kf = StratifiedKFold(n_splits=k,random_state=seed, shuffle=True)
label_count = {'PERFORMANCE':2,'Label_num_level_1':5,'Label_num_level_2':16}
for label, count in label_count.items():
  x=training_data
  y=training_data[label]
  metrics=[]
  i=1
  for train_index, val_index in kf.split(x,y):
    train_texts, val_texts =x['Concept'].iloc[train_index],x['Concept'].iloc[val_index]
    train_labels, val_labels =y.iloc[train_index],y.iloc[val_index]
    list_text = []
    for words in train_texts:
      list_text.append(words)

    # Preprocessing data to feed into MultinomialNB Classifier
    # Tried ngram_range=[1,1],[2,2],[3,3]
    count_vect = CountVectorizer(ngram_range=[1,3])
    tfidf_transformer=TfidfTransformer()

    # Pipeline using CountVectorizer and TfidfTransformer
    pipeline = Pipeline([('count_vect',count_vect),('tfidf_transformer',tfidf_transformer)])
    pipeline_output = pipeline.fit_transform(list_text)
    param_grid_ = {'C': [1e-5, 1e-3, 1e-1, 1e0, 1e1, 1e2]}
    clf = GridSearchCV(LogisticRegression(solver='lbfgs', max_iter=1000, class_weight='balanced'), cv=5, param_grid=param_grid_).fit(pipeline_output, train_labels)
    # Evaluation Model performance on the Test data
    sentence_list = val_texts.to_list()
    pipeline_test = Pipeline([('count_vect',count_vect),('tfidf_transformer',tfidf_transformer)])
    pipeline_test_output = pipeline_test.transform(sentence_list)
    predicted = clf.predict(pipeline_test_output)

    # #Predicting Causal variables into Performance and Non Performance groups
    # variables_list = merged_causal_statements['Variables'].to_list()
    # pipeline_causal=pipeline_test.transform(variables_list)
    # predicted_causal = clf.predict(pipeline_causal)
    # classification_prediction_causal_list =[]
    # for variable, classification in zip(variables_list, predicted_causal):
    #   classification_prediction_causal_list.append(classification)

    classification_prediction_list =[]
    for sentence, classification in zip(sentence_list, predicted):
      classification_prediction_list.append(classification)
    # Naive_Bayes Model metrics
    classification_list = val_labels.to_list()
    indices = val_index
    training_data.loc[indices, "Logistic_{}_Predictions".format(label)]=classification_prediction_list
    # merged_causal_statements["Logistic_{}_Predictions_{}".format(label,i)]=classification_prediction_causal_list
    i = i+1


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in 

In [ ]:
print("Logistic")
print("PERFORMANCE")
print(classification_report(training_data['PERFORMANCE'], training_data['Logistic_PERFORMANCE_Predictions'], zero_division = 1))
print("Label_num_level_1")
print(classification_report(training_data['Label_num_level_1'], training_data['Logistic_Label_num_level_1_Predictions'], zero_division = 1))
print("Label_num_level_2")
print(classification_report(training_data['Label_num_level_2'], training_data['Logistic_Label_num_level_2_Predictions'], zero_division = 1))

Logistic
PERFORMANCE
              precision    recall  f1-score   support

           0       0.85      0.85      0.85       421
           1       0.77      0.77      0.77       268

    accuracy                           0.82       689
   macro avg       0.81      0.81      0.81       689
weighted avg       0.82      0.82      0.82       689

Label_num_level_1
              precision    recall  f1-score   support

           0       0.61      0.70      0.65        43
           1       0.68      0.75      0.71        55
           2       0.67      0.87      0.76        87
           3       0.92      0.81      0.87       421
           4       0.76      0.87      0.81        83

    accuracy                           0.82       689
   macro avg       0.73      0.80      0.76       689
weighted avg       0.83      0.82      0.82       689

Label_num_level_2
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         4
           1      

In [ ]:
k=5
seed = 3
kf = StratifiedKFold(n_splits=k,random_state=seed, shuffle=True)
label_count = {'PERFORMANCE':2,'Label_num_level_1':5,'Label_num_level_2':16}
for label, count in label_count.items():
  x=training_data
  y=training_data[label]
  metrics=[]
  i=1
  for train_index, val_index in kf.split(x,y):
    train_texts, val_texts =x['Concept'].iloc[train_index],x['Concept'].iloc[val_index]
    train_labels, val_labels =y.iloc[train_index],y.iloc[val_index]
    list_text = []
    for words in train_texts:
      list_text.append(words)

    # Preprocessing data to feed into MultinomialNB Classifier
    # Tried ngram_range=[1,1],[2,2],[3,3]
    count_vect = CountVectorizer(ngram_range=[1,3])
    tfidf_transformer=TfidfTransformer()

    # Pipeline using CountVectorizer and TfidfTransformer
    pipeline = Pipeline([('count_vect',count_vect),('tfidf_transformer',tfidf_transformer)])
    pipeline_output = pipeline.fit_transform(list_text)
    param_grid_ = {'C': [1e-5, 1e-3, 1e-1, 1e0, 1e1, 1e2]}
    clf = GridSearchCV(LogisticRegression(solver='lbfgs', max_iter=1000, class_weight='balanced'), cv=5, param_grid=param_grid_).fit(pipeline_output, train_labels)
    # Evaluation Model performance on the Test data
    sentence_list = val_texts.to_list()
    pipeline_test = Pipeline([('count_vect',count_vect),('tfidf_transformer',tfidf_transformer)])
    pipeline_test_output = pipeline_test.transform(sentence_list)
    predicted = clf.predict(pipeline_test_output)

    classification_prediction_list =[]
    for sentence, classification in zip(sentence_list, predicted):
      classification_prediction_list.append(classification)
    # Naive_Bayes Model metrics
    classification_list = val_labels.to_list()
    indices = val_index
    training_data.loc[indices, "Logistic_{}_Predictions".format(label)]=classification_prediction_list
    # merged_causal_statements["Logistic_{}_Predictions_{}".format(label,i)]=classification_prediction_causal_list
    i = i+1


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in 

In [ ]:
print("Logistic")
print("PERFORMANCE")
print(classification_report(training_data['PERFORMANCE'], training_data['Logistic_PERFORMANCE_Predictions'], zero_division = 1))
print("Label_num_level_1")
print(classification_report(training_data['Label_num_level_1'], training_data['Logistic_Label_num_level_1_Predictions'], zero_division = 1))
print("Label_num_level_2")
print(classification_report(training_data['Label_num_level_2'], training_data['Logistic_Label_num_level_2_Predictions'], zero_division = 1))

Logistic
PERFORMANCE
              precision    recall  f1-score   support

           0       0.86      0.84      0.85       421
           1       0.76      0.79      0.78       268

    accuracy                           0.82       689
   macro avg       0.81      0.82      0.81       689
weighted avg       0.82      0.82      0.82       689

Label_num_level_1
              precision    recall  f1-score   support

           0       0.63      0.74      0.68        43
           1       0.67      0.71      0.69        55
           2       0.69      0.82      0.75        87
           3       0.90      0.81      0.85       421
           4       0.73      0.83      0.78        83

    accuracy                           0.80       689
   macro avg       0.72      0.78      0.75       689
weighted avg       0.81      0.80      0.81       689

Label_num_level_2
              precision    recall  f1-score   support

           0       0.20      0.25      0.22         4
           1      

In [ ]:
k=5
seed = 4
kf = StratifiedKFold(n_splits=k,random_state=seed, shuffle=True)
label_count = {'PERFORMANCE':2,'Label_num_level_1':5,'Label_num_level_2':16}
for label, count in label_count.items():
  x=training_data
  y=training_data[label]
  metrics=[]
  i=1
  for train_index, val_index in kf.split(x,y):
    train_texts, val_texts =x['Concept'].iloc[train_index],x['Concept'].iloc[val_index]
    train_labels, val_labels =y.iloc[train_index],y.iloc[val_index]
    list_text = []
    for words in train_texts:
      list_text.append(words)

    # Preprocessing data to feed into MultinomialNB Classifier
    # Tried ngram_range=[1,1],[2,2],[3,3]
    count_vect = CountVectorizer(ngram_range=[1,3])
    tfidf_transformer=TfidfTransformer()

    # Pipeline using CountVectorizer and TfidfTransformer
    pipeline = Pipeline([('count_vect',count_vect),('tfidf_transformer',tfidf_transformer)])
    pipeline_output = pipeline.fit_transform(list_text)
    param_grid_ = {'C': [1e-5, 1e-3, 1e-1, 1e0, 1e1, 1e2]}
    clf = GridSearchCV(LogisticRegression(solver='lbfgs', max_iter=1000, class_weight='balanced'), cv=5, param_grid=param_grid_).fit(pipeline_output, train_labels)
    # Evaluation Model performance on the Test data
    sentence_list = val_texts.to_list()
    pipeline_test = Pipeline([('count_vect',count_vect),('tfidf_transformer',tfidf_transformer)])
    pipeline_test_output = pipeline_test.transform(sentence_list)
    predicted = clf.predict(pipeline_test_output)

    classification_prediction_list =[]
    for sentence, classification in zip(sentence_list, predicted):
      classification_prediction_list.append(classification)
    # Naive_Bayes Model metrics
    classification_list = val_labels.to_list()
    indices = val_index
    training_data.loc[indices, "Logistic_{}_Predictions".format(label)]=classification_prediction_list
    # merged_causal_statements["Logistic_{}_Predictions_{}".format(label,i)]=classification_prediction_causal_list
    i = i+1


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in 

In [ ]:
print("Logistic")
print("PERFORMANCE")
print(classification_report(training_data['PERFORMANCE'], training_data['Logistic_PERFORMANCE_Predictions'], zero_division = 1))
print("Label_num_level_1")
print(classification_report(training_data['Label_num_level_1'], training_data['Logistic_Label_num_level_1_Predictions'], zero_division = 1))
print("Label_num_level_2")
print(classification_report(training_data['Label_num_level_2'], training_data['Logistic_Label_num_level_2_Predictions'], zero_division = 1))

Logistic
PERFORMANCE
              precision    recall  f1-score   support

           0       0.85      0.85      0.85       421
           1       0.77      0.77      0.77       268

    accuracy                           0.82       689
   macro avg       0.81      0.81      0.81       689
weighted avg       0.82      0.82      0.82       689

Label_num_level_1
              precision    recall  f1-score   support

           0       0.69      0.72      0.70        43
           1       0.69      0.67      0.68        55
           2       0.68      0.87      0.77        87
           3       0.91      0.83      0.87       421
           4       0.74      0.83      0.78        83

    accuracy                           0.82       689
   macro avg       0.74      0.79      0.76       689
weighted avg       0.83      0.82      0.82       689

Label_num_level_2
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         4
           1      

In [ ]:
k=5
seed = 1
kf = StratifiedKFold(n_splits=k,random_state=seed, shuffle=True)
label_count = {'PERFORMANCE':2,'Label_num_level_1':5,'Label_num_level_2':16}
for label, count in label_count.items():
  x=training_data
  y=training_data[label]
  metrics=[]
  i=1
  for train_index, val_index in kf.split(x,y):
    train_texts, val_texts =x['Concept'].iloc[train_index],x['Concept'].iloc[val_index]
    train_labels, val_labels =y.iloc[train_index],y.iloc[val_index]
    list_text = []
    for words in train_texts:
      list_text.append(words)

    # Preprocessing data to feed into MultinomialNB Classifier
    # Tried ngram_range=[1,1],[2,2],[3,3]
    count_vect = CountVectorizer(ngram_range=[1,2])
    tfidf_transformer=TfidfTransformer()

    # Pipeline using CountVectorizer and TfidfTransformer
    pipeline = Pipeline([('count_vect',count_vect),('tfidf_transformer',tfidf_transformer)])
    pipeline_output = pipeline.fit_transform(list_text)
    param_grid_ = {'C': [1e-5, 1e-3, 1e-1, 1e0, 1e1, 1e2]}
    clf = GridSearchCV(LogisticRegression(solver='lbfgs', max_iter=1000, class_weight='balanced'), cv=5, param_grid=param_grid_).fit(pipeline_output, train_labels)
    # Evaluation Model performance on the Test data
    sentence_list = val_texts.to_list()
    pipeline_test = Pipeline([('count_vect',count_vect),('tfidf_transformer',tfidf_transformer)])
    pipeline_test_output = pipeline_test.transform(sentence_list)
    predicted = clf.predict(pipeline_test_output)

    classification_prediction_list =[]
    for sentence, classification in zip(sentence_list, predicted):
      classification_prediction_list.append(classification)
    # Naive_Bayes Model metrics
    classification_list = val_labels.to_list()
    indices = val_index
    training_data.loc[indices, "Logistic_{}_Predictions".format(label)]=classification_prediction_list
    # merged_causal_statements["Logistic_{}_Predictions_{}".format(label,i)]=classification_prediction_causal_list
    i = i+1


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in 

In [ ]:
print("Logistic")
print("PERFORMANCE")
print(classification_report(training_data['PERFORMANCE'], training_data['Logistic_PERFORMANCE_Predictions'], zero_division = 1))
print("Label_num_level_1")
print(classification_report(training_data['Label_num_level_1'], training_data['Logistic_Label_num_level_1_Predictions'], zero_division = 1))
print("Label_num_level_2")
print(classification_report(training_data['Label_num_level_2'], training_data['Logistic_Label_num_level_2_Predictions'], zero_division = 1))

Logistic
PERFORMANCE
              precision    recall  f1-score   support

           0       0.87      0.85      0.86       421
           1       0.77      0.79      0.78       268

    accuracy                           0.83       689
   macro avg       0.82      0.82      0.82       689
weighted avg       0.83      0.83      0.83       689

Label_num_level_1
              precision    recall  f1-score   support

           0       0.70      0.72      0.71        43
           1       0.80      0.73      0.76        55
           2       0.78      0.80      0.79        87
           3       0.89      0.87      0.88       421
           4       0.72      0.82      0.77        83

    accuracy                           0.84       689
   macro avg       0.78      0.79      0.78       689
weighted avg       0.84      0.84      0.84       689

Label_num_level_2
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         4
           1      

In [ ]:
k=5
seed = 1
kf = StratifiedKFold(n_splits=k,random_state=seed, shuffle=True)
label_count = {'PERFORMANCE':2,'Label_num_level_1':5,'Label_num_level_2':16}
for label, count in label_count.items():
  x=training_data
  y=training_data[label]
  metrics=[]
  i=1
  for train_index, val_index in kf.split(x,y):
    train_texts, val_texts =x['Concept'].iloc[train_index],x['Concept'].iloc[val_index]
    train_labels, val_labels =y.iloc[train_index],y.iloc[val_index]
    list_text = []
    for words in train_texts:
      list_text.append(words)

    # Preprocessing data to feed into MultinomialNB Classifier
    # Tried ngram_range=[1,1],[2,2],[3,3]
    count_vect = CountVectorizer(ngram_range=[1,1])
    tfidf_transformer=TfidfTransformer()

    # Pipeline using CountVectorizer and TfidfTransformer
    pipeline = Pipeline([('count_vect',count_vect),('tfidf_transformer',tfidf_transformer)])
    pipeline_output = pipeline.fit_transform(list_text)
    param_grid_ = {'C': [1e-5, 1e-3, 1e-1, 1e0, 1e1, 1e2]}
    clf = GridSearchCV(LogisticRegression(solver='lbfgs', max_iter=1000, class_weight='balanced'), cv=5, param_grid=param_grid_).fit(pipeline_output, train_labels)
    # Evaluation Model performance on the Test data
    sentence_list = val_texts.to_list()
    pipeline_test = Pipeline([('count_vect',count_vect),('tfidf_transformer',tfidf_transformer)])
    pipeline_test_output = pipeline_test.transform(sentence_list)
    predicted = clf.predict(pipeline_test_output)

    classification_prediction_list =[]
    for sentence, classification in zip(sentence_list, predicted):
      classification_prediction_list.append(classification)
    # Naive_Bayes Model metrics
    classification_list = val_labels.to_list()
    indices = val_index
    training_data.loc[indices, "Logistic_{}_Predictions".format(label)]=classification_prediction_list
    # merged_causal_statements["Logistic_{}_Predictions_{}".format(label,i)]=classification_prediction_causal_list
    i = i+1


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in 

In [ ]:
print("Logistic")
print("PERFORMANCE")
print(classification_report(training_data['PERFORMANCE'], training_data['Logistic_PERFORMANCE_Predictions'], zero_division = 1))
print("Label_num_level_1")
print(classification_report(training_data['Label_num_level_1'], training_data['Logistic_Label_num_level_1_Predictions'], zero_division = 1))
print("Label_num_level_2")
print(classification_report(training_data['Label_num_level_2'], training_data['Logistic_Label_num_level_2_Predictions'], zero_division = 1))

Logistic
PERFORMANCE
              precision    recall  f1-score   support

           0       0.86      0.84      0.85       421
           1       0.75      0.78      0.77       268

    accuracy                           0.82       689
   macro avg       0.81      0.81      0.81       689
weighted avg       0.82      0.82      0.82       689

Label_num_level_1
              precision    recall  f1-score   support

           0       0.70      0.74      0.72        43
           1       0.71      0.71      0.71        55
           2       0.76      0.82      0.78        87
           3       0.90      0.85      0.87       421
           4       0.73      0.82      0.77        83

    accuracy                           0.83       689
   macro avg       0.76      0.79      0.77       689
weighted avg       0.83      0.83      0.83       689

Label_num_level_2
              precision    recall  f1-score   support

           0       0.40      0.50      0.44         4
           1      

In [ ]:
type(training_data['PERFORMANCE'])

pandas.core.series.Series

In [ ]:
training_data['PERFORMANCE'].to_excel('golddata_LR.xlsx')

In [ ]:
training_data['Logistic_PERFORMANCE_Predictions'].to_excel('predicted_performance_LR.xlsx')

In [ ]:
training_data['Label_num_level_1'].to_excel('golddata_level1_LR.xlsx')

In [ ]:
training_data['Logistic_Label_num_level_1_Predictions'].to_excel('predicted_level1_LR.xlsx')

In [ ]:
training_data['Label_num_level_2'].to_excel('golddata_level2_LR.xlsx')

In [ ]:
train_data = pd.concat([training_data['PERFORMANCE'],training_data['Logistic_PERFORMANCE_Predictions']],axis=1)

In [ ]:
train_data_level1 = pd.concat([training_data['Label_num_level_1'], training_data['Logistic_Label_num_level_1_Predictions']],axis=1)

In [ ]:
train_data_level2 = pd.concat([training_data['Label_num_level_2'],training_data['Logistic_Label_num_level_2_Predictions']],axis=1)

In [ ]:
train_final = pd.concat([train_data,train_data_level1,train_data_level2],axis=1)

In [ ]:
train_final

,PERFORMANCE,Logistic_PERFORMANCE_Predictions,Label_num_level_1,Logistic_Label_num_level_1_Predictions,Label_num_level_2,Logistic_Label_num_level_2_Predictions
0,1,1.0,0,0.0,1,1.0
1,1,1.0,2,2.0,7,7.0
2,1,1.0,0,0.0,1,1.0
3,0,0.0,3,3.0,11,11.0
4,1,0.0,1,1.0,4,11.0
...,...,...,...,...,...,...
684,1,1.0,1,1.0,4,4.0
685,1,1.0,2,2.0,7,7.0
686,0,0.0,3,3.0,11,11.0
687,0,0.0,3,3.0,11,11.0


In [ ]:
train_final.to_excel('erro_analysis_LR_final.xlsx')

In [ ]:
!pip install tensorflow==1.15
!pip install "tensorflow_hub>=0.6.0"
!pip3 install tensorflow_text==1.15

     |████████████████████████████████| 412.3 MB 26 kB/s 
     |████████████████████████████████| 3.8 MB 42.4 MB/s 
     |████████████████████████████████| 50 kB 7.1 MB/s 
     |████████████████████████████████| 503 kB 29.4 MB/s 
  Created wheel for gast: filename=gast-0.2.2-py3-none-any.whl size=7554 sha256=25ab87ccb98737acb61e30f885de0e3660aaea0611fb5e03682f70b53afdc106
  Stored in directory: /root/.cache/pip/wheels/21/7f/02/420f32a803f7d0967b48dd823da3f558c5166991bfd204eef3
Successfully built gast
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.7.0
    Uninstalling tensorflow-estimator-2.7.0:
      Successfully uninstalled tensorflow-estimator-2.7.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.7.0
    Uninstalling tensorboard-2.7.0:
      Successfully uninstalled tensorboard-2.7.0
  Attempting uninstall: gast
    Found existing installation: gast 0.4.0
    Uninstalling gast-0.4.0:
      Succes

In [ ]:
from sklearn.linear_model import LogisticRegression
import sklearn.model_selection
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline, FeatureUnion
from pandas import read_csv
import pandas as pd
import math
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
import numpy as np

In [ ]:
import tensorflow_hub as hub
import tensorflow as tf

print("TF version:", tf.__version__)
print("Hub version:", hub.__version__)
print("GPU is", "available" if tf.test.is_gpu_available() else "NOT AVAILABLE")

elmo = hub.Module("https://tfhub.dev/google/elmo/2", trainable=True)

TF version: 1.15.0
Hub version: 0.12.0
GPU is NOT AVAILABLE


In [ ]:
def elmo_vectors(x):
  embeddings = elmo(x.tolist(), signature="default", as_dict=True)["elmo"]

  with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    sess.run(tf.tables_initializer())
    # return average of ELMo features
    return sess.run(tf.reduce_mean(embeddings,1))

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
k=5
seed = 1
kf = StratifiedKFold(n_splits=k,random_state=seed, shuffle=True)
label_count = {'PERFORMANCE':2,'Label_num_level_1':5,'Label_num_level_2':16}
for label, count in label_count.items():
  x=training_data
  y=training_data[label]
  metrics=[]
  i=1
  for train_index, val_index in kf.split(x,y):
    train_texts, val_texts =x['Concept'].iloc[train_index],x['Concept'].iloc[val_index]
    train_labels, val_labels =y.iloc[train_index],y.iloc[val_index]
    list_text = []
    for words in train_texts:
      list_text.append(words)


    # Preprocessing data to feed into MultinomialNB Classifier
    # Tried ngram_range=[1,1],[2,2],[3,3]
    count_vect = CountVectorizer(ngram_range=[1,3])
    list_train = [train_texts[i:i+100] for i in range(0,train_texts.shape[0],100)]
    list_test = [val_texts[i:i+100] for i in range(0,val_texts.shape[0],100)]

    elmo_train = [elmo_vectors(x) for x in list_train]
    elmo_test = [elmo_vectors(x) for x in list_test]

    elmo_train_new = np.concatenate(elmo_train, axis = 0)
    elmo_test_new = np.concatenate(elmo_test, axis = 0)

    param_grid_ = {'C': [1e-5, 1e-3, 1e-1, 1e0, 1e1, 1e2]}
    clf = GridSearchCV(LogisticRegression(solver='saga', max_iter=5000, class_weight='balanced'), cv=5, param_grid=param_grid_).fit(elmo_train_new, train_labels)
    sentence_list = val_texts.to_list()
    prediction_grid = clf.predict(elmo_test_new)
    classification_prediction_list =[]
    for sentence, classification in zip(sentence_list, prediction_grid):
      classification_prediction_list.append(classification)
    print(classification_prediction_list)
    # Naive_Bayes Model metrics
    classification_list = val_labels.to_list()
    indices = val_index
    training_data.loc[indices, "Logistic_{}_Predictions".format(label)]=classification_prediction_list
    i = i+1

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


...

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag

[3, 11, 7, 9, 9, 2, 9, 11, 11, 11, 11, 11, 11, 2, 13, 3, 3, 11, 11, 13, 14, 3, 11, 11, 11, 11, 4, 7, 11, 6, 11, 11, 2, 9, 11, 11, 11, 0, 9, 9, 11, 11, 6, 5, 5, 3, 14, 15, 2, 3, 11, 15, 11, 11, 11, 9, 7, 11, 11, 11, 11, 9, 11, 7, 9, 11, 11, 11, 11, 3, 11, 9, 3, 7, 7, 3, 11, 11, 15, 7, 4, 11, 6, 11, 11, 11, 11, 4, 4, 11, 11, 11, 11, 11, 11, 13, 11, 11, 11, 11, 13, 14, 11, 11, 11, 11, 3, 11, 11, 11, 11, 11, 11, 11, 13, 11, 11, 12, 6, 13, 11, 11, 11, 7, 11, 11, 15, 11, 11, 11, 11, 11, 4, 11, 11, 14, 12]


In [ ]:
print("Logistic")
print("PERFORMANCE")
print(classification_report(training_data['PERFORMANCE'], training_data['Logistic_PERFORMANCE_Predictions'], zero_division = 1))
print("Label_num_level_1")
print(classification_report(training_data['Label_num_level_1'], training_data['Logistic_Label_num_level_1_Predictions'], zero_division = 1))
print("Label_num_level_2")
print(classification_report(training_data['Label_num_level_2'], training_data['Logistic_Label_num_level_2_Predictions'], zero_division = 1))

Logistic
PERFORMANCE
              precision    recall  f1-score   support

           0       0.82      0.83      0.83       421
           1       0.73      0.72      0.72       268

    accuracy                           0.79       689
   macro avg       0.78      0.78      0.78       689
weighted avg       0.79      0.79      0.79       689

Label_num_level_1
              precision    recall  f1-score   support

           0       0.66      0.58      0.62        43
           1       0.66      0.67      0.67        55
           2       0.66      0.74      0.70        87
           3       0.82      0.81      0.82       421
           4       0.68      0.70      0.69        83

    accuracy                           0.76       689
   macro avg       0.70      0.70      0.70       689
weighted avg       0.76      0.76      0.76       689

Label_num_level_2
              precision    recall  f1-score   support

           0       0.50      0.75      0.60         4
           1      

In [ ]:
k=5
seed = 2
kf = StratifiedKFold(n_splits=k,random_state=seed, shuffle=True)
label_count = {'PERFORMANCE':2,'Label_num_level_1':5,'Label_num_level_2':16}
for label, count in label_count.items():
  x=training_data
  y=training_data[label]
  metrics=[]
  i=1
  for train_index, val_index in kf.split(x,y):
    train_texts, val_texts =x['Concept'].iloc[train_index],x['Concept'].iloc[val_index]
    train_labels, val_labels =y.iloc[train_index],y.iloc[val_index]
    list_text = []
    for words in train_texts:
      list_text.append(words)


    # Preprocessing data to feed into MultinomialNB Classifier
    # Tried ngram_range=[1,1],[2,2],[3,3]
    count_vect = CountVectorizer(ngram_range=[1,3])
    list_train = [train_texts[i:i+100] for i in range(0,train_texts.shape[0],100)]
    list_test = [val_texts[i:i+100] for i in range(0,val_texts.shape[0],100)]

    elmo_train = [elmo_vectors(x) for x in list_train]
    elmo_test = [elmo_vectors(x) for x in list_test]

    elmo_train_new = np.concatenate(elmo_train, axis = 0)
    elmo_test_new = np.concatenate(elmo_test, axis = 0)

    param_grid_ = {'C': [1e-5, 1e-3, 1e-1, 1e0, 1e1, 1e2]}
    clf = GridSearchCV(LogisticRegression(solver='saga', max_iter=5000, class_weight='balanced'), cv=5, param_grid=param_grid_).fit(elmo_train_new, train_labels)
    sentence_list = val_texts.to_list()
    prediction_grid = clf.predict(elmo_test_new)
    classification_prediction_list =[]
    for sentence, classification in zip(sentence_list, prediction_grid):
      classification_prediction_list.append(classification)
    print(classification_prediction_list)
    # Naive_Bayes Model metrics
    classification_list = val_labels.to_list()
    indices = val_index
    training_data.loc[indices, "Logistic_{}_Predictions".format(label)]=classification_prediction_list
    i = i+1

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


...

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag

[13, 11, 11, 7, 11, 11, 11, 11, 11, 11, 9, 2, 7, 11, 11, 11, 11, 7, 13, 11, 11, 13, 11, 11, 11, 11, 10, 11, 13, 11, 11, 11, 11, 2, 7, 11, 11, 11, 7, 3, 13, 11, 11, 13, 11, 11, 11, 2, 11, 11, 15, 14, 11, 11, 11, 11, 11, 14, 11, 11, 11, 11, 9, 11, 4, 6, 11, 11, 11, 3, 3, 6, 4, 3, 11, 11, 7, 4, 11, 9, 11, 11, 7, 2, 11, 11, 11, 4, 15, 11, 11, 11, 11, 11, 7, 6, 0, 11, 11, 11, 10, 11, 13, 6, 13, 7, 13, 11, 11, 11, 11, 11, 11, 11, 11, 11, 12, 11, 11, 13, 7, 11, 11, 15, 11, 11, 11, 13, 11, 11, 7, 15, 11, 11, 7, 11, 12]


In [ ]:
print("Logistic")
print("PERFORMANCE")
print(classification_report(training_data['PERFORMANCE'], training_data['Logistic_PERFORMANCE_Predictions'], zero_division = 1))
print("Label_num_level_1")
print(classification_report(training_data['Label_num_level_1'], training_data['Logistic_Label_num_level_1_Predictions'], zero_division = 1))
print("Label_num_level_2")
print(classification_report(training_data['Label_num_level_2'], training_data['Logistic_Label_num_level_2_Predictions'], zero_division = 1))

Logistic
PERFORMANCE
              precision    recall  f1-score   support

           0       0.84      0.81      0.82       421
           1       0.72      0.75      0.73       268

    accuracy                           0.79       689
   macro avg       0.78      0.78      0.78       689
weighted avg       0.79      0.79      0.79       689

Label_num_level_1
              precision    recall  f1-score   support

           0       0.77      0.56      0.65        43
           1       0.64      0.58      0.61        55
           2       0.69      0.62      0.65        87
           3       0.80      0.85      0.83       421
           4       0.72      0.72      0.72        83

    accuracy                           0.77       689
   macro avg       0.73      0.67      0.69       689
weighted avg       0.76      0.77      0.76       689

Label_num_level_2
              precision    recall  f1-score   support

           0       0.50      0.50      0.50         4
           1      

In [ ]:
!pip install spacy

In [ ]:
!python -m spacy download en_core_web_md

     |████████████████████████████████| 96.4 MB 58.0 MB/s 
  Created wheel for en-core-web-md: filename=en_core_web_md-2.2.5-py3-none-any.whl size=98051301 sha256=2b5a88a0ce5c02c10a7a8944a4a36f5aca0963432ef12ad98a9d11c2e0d8a55e
  Stored in directory: /tmp/pip-ephem-wheel-cache-pnu1g3tl/wheels/69/c5/b8/4f1c029d89238734311b3269762ab2ee325a42da2ce8edb997
Successfully built en-core-web-md
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_md')


In [ ]:
import spacy
# Load the spacy model
nlp = spacy.load('en_core_web_md')

In [ ]:
# train_embedding = []
# for train in train_final:
#   train_embedding.append(nlp(train))

In [ ]:
k=5
seed = 2
kf = StratifiedKFold(n_splits=k,random_state=seed, shuffle=True)
label_count = {'PERFORMANCE':2,'Label_num_level_1':5,'Label_num_level_2':16}
for label, count in label_count.items():
  x=training_data
  y=training_data[label]
  metrics=[]
  i=1
  for train_index, val_index in kf.split(x,y):
    train_texts, val_texts =x['Concept'].iloc[train_index],x['Concept'].iloc[val_index]
    train_labels, val_labels =y.iloc[train_index],y.iloc[val_index]
    list_text = []
    for words in train_texts:
      list_text.append(words)

    train_embedding = []
    for train in train_texts:
      train_embedding.append(nlp(train))

    train_embedding_final =[]
    for i in train_embedding:
      train_embedding_final.append(i.vector)

    test_embedding = []
    for test in val_texts:
      test_embedding.append(nlp(test))

    test_embedding_final = []
    for j in test_embedding:
      test_embedding_final.append(j.vector)

    param_grid_ = {'C': [1e-5, 1e-3, 1e-1, 1e0, 1e1, 1e2]}
    clf = GridSearchCV(LogisticRegression(solver='saga', max_iter=5000, class_weight='balanced'), cv=5, param_grid=param_grid_).fit(train_embedding_final, train_labels)
    sentence_list = val_texts.to_list()
    prediction_grid = clf.predict(test_embedding_final)
    classification_prediction_list =[]
    for sentence, classification in zip(sentence_list, prediction_grid):
      classification_prediction_list.append(classification)
    print(classification_prediction_list)
    # Naive_Bayes Model metrics
    classification_list = val_labels.to_list()
    indices = val_index
    training_data.loc[indices, "Logistic_{}_Predictions".format(label)]=classification_prediction_list
    #i = i+1


[1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1]
[1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1]
[1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


[2, 2, 3, 3, 4, 3, 1, 0, 0, 3, 2, 2, 3, 4, 3, 4, 4, 4, 0, 2, 3, 2, 4, 3, 3, 1, 2, 3, 2, 3, 3, 2, 4, 3, 3, 3, 3, 3, 2, 3, 3, 0, 3, 3, 4, 2, 3, 3, 2, 3, 3, 3, 4, 0, 3, 4, 3, 3, 3, 4, 1, 3, 3, 2, 3, 1, 3, 3, 4, 3, 2, 1, 3, 3, 2, 3, 1, 3, 3, 3, 3, 3, 3, 2, 4, 4, 1, 3, 2, 3, 1, 3, 3, 1, 2, 3, 3, 3, 3, 4, 4, 2, 3, 3, 4, 4, 3, 3, 3, 3, 2, 4, 2, 0, 3, 1, 3, 3, 3, 0, 3, 3, 1, 2, 3, 3, 3, 2, 2, 3, 3, 2, 2, 2, 3, 3, 3, 4]
[0, 1, 1, 0, 3, 2, 0, 3, 3, 4, 3, 4, 4, 3, 3, 1, 4, 3, 0, 4, 3, 3, 4, 3, 3, 3, 3, 3, 4, 2, 3, 3, 3, 4, 0, 1, 2, 3, 3, 3, 2, 2, 3, 3, 4, 2, 4, 0, 4, 3, 3, 1, 1, 2, 3, 3, 1, 2, 3, 2, 3, 1, 0, 0, 3, 2, 0, 0, 1, 4, 3, 1, 3, 3, 3, 2, 2, 3, 1, 3, 3, 4, 4, 3, 2, 3, 3, 3, 4, 3, 3, 3, 3, 3, 4, 4, 3, 4, 4, 3, 3, 3, 1, 3, 0, 3, 4, 3, 3, 2, 4, 4, 2, 3, 3, 0, 3, 2, 1, 4, 3, 2, 3, 2, 0, 3, 0, 3, 3, 3, 1, 3, 3, 3, 4, 3, 3, 2]
[2, 3, 4, 3, 3, 2, 4, 1, 3, 4, 3, 3, 3, 3, 3, 3, 4, 4, 3, 3, 3, 3, 3, 0, 3, 1, 4, 2, 1, 4, 3, 3, 2, 4, 3, 0, 2, 0, 1, 3, 3, 1, 1, 4, 4, 2, 2, 3, 0, 3, 3, 3, 2, 3, 2, 3, 3

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


[0, 4, 2, 3, 3, 3, 4, 2, 3, 3, 3, 4, 4, 3, 4, 3, 3, 3, 1, 2, 2, 2, 1, 3, 3, 4, 3, 3, 3, 0, 0, 3, 3, 2, 1, 1, 0, 3, 3, 3, 4, 4, 3, 1, 3, 4, 3, 4, 3, 0, 3, 3, 3, 2, 3, 2, 3, 1, 2, 3, 2, 1, 3, 3, 1, 3, 1, 1, 2, 2, 3, 3, 0, 0, 3, 3, 2, 0, 3, 3, 3, 3, 3, 4, 2, 3, 4, 3, 4, 2, 3, 2, 3, 3, 3, 3, 3, 1, 3, 3, 4, 3, 3, 4, 3, 1, 3, 4, 1, 3, 3, 3, 4, 2, 0, 3, 3, 3, 4, 4, 2, 0, 2, 0, 3, 3, 2, 1, 3, 3, 3, 3, 4, 2, 3, 2, 3, 4]


...

[1, 6, 11, 11, 9, 13, 11, 11, 11, 6, 13, 11, 10, 11, 11, 6, 11, 13, 6, 11, 11, 11, 13, 2, 11, 11, 11, 7, 6, 11, 11, 11, 3, 6, 4, 11, 11, 4, 11, 11, 6, 5, 11, 5, 3, 11, 11, 11, 15, 11, 15, 12, 2, 11, 11, 11, 11, 11, 11, 11, 7, 4, 11, 7, 11, 7, 11, 11, 6, 11, 7, 7, 3, 11, 15, 2, 11, 3, 11, 13, 7, 15, 9, 6, 15, 11, 11, 0, 13, 11, 11, 11, 11, 11, 7, 11, 13, 11, 13, 7, 11, 11, 11, 11, 3, 13, 12, 7, 6, 7, 14, 9, 11, 11, 11, 11, 11, 11, 8, 3, 11, 11, 2, 11, 11, 11, 11, 10, 11, 11, 3, 11, 7, 7, 11, 7, 4, 13]


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not conve

[7, 1, 15, 2, 6, 11, 15, 11, 11, 9, 11, 14, 6, 11, 7, 11, 11, 7, 13, 6, 11, 6, 11, 11, 11, 13, 3, 7, 11, 11, 2, 9, 11, 11, 11, 11, 13, 11, 11, 7, 7, 11, 11, 15, 3, 11, 14, 11, 6, 6, 11, 13, 11, 15, 2, 9, 11, 3, 4, 7, 3, 11, 10, 6, 0, 11, 11, 9, 7, 11, 2, 4, 13, 7, 11, 4, 11, 15, 7, 11, 9, 13, 9, 11, 13, 13, 11, 11, 11, 11, 11, 11, 11, 11, 15, 13, 6, 11, 11, 11, 11, 11, 14, 11, 11, 15, 11, 11, 11, 3, 11, 7, 13, 13, 7, 3, 11, 15, 11, 7, 11, 4, 11, 11, 3, 11, 13, 11, 11, 11, 11, 13, 11, 7, 11, 11, 9, 11]


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not conve

[13, 11, 11, 7, 6, 11, 11, 11, 13, 11, 11, 2, 9, 11, 11, 11, 11, 3, 13, 11, 1, 13, 7, 11, 11, 11, 7, 11, 13, 7, 15, 11, 11, 2, 7, 11, 3, 11, 7, 11, 14, 11, 11, 13, 11, 7, 11, 2, 11, 11, 15, 14, 11, 11, 11, 11, 11, 14, 11, 11, 11, 11, 9, 11, 5, 7, 11, 11, 11, 3, 9, 6, 4, 3, 11, 11, 7, 4, 11, 11, 15, 11, 7, 2, 11, 11, 3, 4, 11, 3, 11, 11, 6, 11, 7, 6, 11, 11, 13, 11, 10, 11, 13, 15, 11, 7, 13, 11, 11, 11, 11, 11, 11, 11, 11, 11, 12, 11, 11, 13, 7, 11, 11, 15, 11, 7, 11, 13, 11, 4, 7, 11, 11, 11, 7, 11, 11]


In [ ]:
print("Logistic")
print("PERFORMANCE")
print(classification_report(training_data['PERFORMANCE'], training_data['Logistic_PERFORMANCE_Predictions'], zero_division = 1))
print("Label_num_level_1")
print(classification_report(training_data['Label_num_level_1'], training_data['Logistic_Label_num_level_1_Predictions'], zero_division = 1))
print("Label_num_level_2")
print(classification_report(training_data['Label_num_level_2'], training_data['Logistic_Label_num_level_2_Predictions'], zero_division = 1))

Logistic
PERFORMANCE
              precision    recall  f1-score   support

           0       0.82      0.78      0.80       421
           1       0.68      0.74      0.70       268

    accuracy                           0.76       689
   macro avg       0.75      0.76      0.75       689
weighted avg       0.77      0.76      0.76       689

Label_num_level_1
              precision    recall  f1-score   support

           0       0.60      0.72      0.65        43
           1       0.60      0.76      0.67        55
           2       0.60      0.82      0.69        87
           3       0.89      0.73      0.80       421
           4       0.72      0.87      0.79        83

    accuracy                           0.76       689
   macro avg       0.68      0.78      0.72       689
weighted avg       0.79      0.76      0.77       689

Label_num_level_2
              precision    recall  f1-score   support

           0       0.20      0.25      0.22         4
           1      

In [ ]:
k=5
seed = 1
kf = StratifiedKFold(n_splits=k,random_state=seed, shuffle=True)
label_count = {'PERFORMANCE':2,'Label_num_level_1':5,'Label_num_level_2':16}
for label, count in label_count.items():
  x=training_data
  y=training_data[label]
  metrics=[]
  i=1
  for train_index, val_index in kf.split(x,y):
    train_texts, val_texts =x['Concept'].iloc[train_index],x['Concept'].iloc[val_index]
    train_labels, val_labels =y.iloc[train_index],y.iloc[val_index]
    list_text = []
    for words in train_texts:
      list_text.append(words)

    train_embedding = []
    for train in train_texts:
      train_embedding.append(nlp(train))

    train_embedding_final =[]
    for i in train_embedding:
      train_embedding_final.append(i.vector)

    test_embedding = []
    for test in val_texts:
      test_embedding.append(nlp(test))

    test_embedding_final = []
    for j in test_embedding:
      test_embedding_final.append(j.vector)

    param_grid_ = {'C': [1e-5, 1e-3, 1e-1, 1e0, 1e1, 1e2]}
    clf = GridSearchCV(LogisticRegression(solver='saga', max_iter=5000, class_weight='balanced'), cv=5, param_grid=param_grid_).fit(train_embedding_final, train_labels)
    sentence_list = val_texts.to_list()
    prediction_grid = clf.predict(test_embedding_final)
    classification_prediction_list =[]
    for sentence, classification in zip(sentence_list, prediction_grid):
      classification_prediction_list.append(classification)
    print(classification_prediction_list)
    # Naive_Bayes Model metrics
    classification_list = val_labels.to_list()
    indices = val_index
    training_data.loc[indices, "Logistic_{}_Predictions".format(label)]=classification_prediction_list
    #i = i+1

[1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0]
[1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1]
[1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


[0, 0, 3, 3, 2, 3, 3, 3, 4, 3, 3, 3, 2, 3, 4, 3, 3, 1, 3, 3, 0, 3, 4, 2, 1, 3, 3, 4, 4, 3, 3, 2, 3, 4, 2, 3, 3, 2, 2, 1, 2, 3, 3, 3, 3, 4, 3, 4, 4, 2, 2, 3, 3, 0, 3, 3, 3, 3, 3, 2, 3, 3, 2, 3, 3, 3, 3, 3, 4, 1, 3, 0, 2, 2, 3, 3, 3, 3, 3, 3, 3, 0, 3, 3, 3, 4, 3, 0, 3, 3, 3, 4, 3, 2, 2, 3, 3, 3, 1, 4, 4, 4, 3, 2, 3, 2, 4, 2, 3, 3, 3, 2, 2, 3, 3, 3, 3, 3, 1, 4, 3, 2, 3, 0, 3, 0, 3, 0, 3, 3, 3, 3, 3, 4, 2, 3, 2, 4]


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


[2, 1, 4, 3, 2, 3, 0, 3, 3, 2, 3, 3, 3, 3, 4, 4, 3, 3, 1, 2, 3, 2, 4, 1, 3, 3, 3, 0, 3, 3, 0, 3, 2, 1, 3, 3, 1, 4, 4, 4, 3, 3, 4, 3, 1, 3, 3, 4, 4, 3, 3, 3, 3, 3, 3, 2, 0, 1, 1, 1, 3, 4, 3, 4, 3, 3, 4, 2, 3, 2, 3, 2, 0, 3, 3, 2, 2, 1, 1, 3, 4, 3, 3, 2, 3, 3, 3, 3, 3, 3, 3, 3, 2, 3, 3, 3, 4, 3, 1, 3, 3, 1, 3, 0, 4, 2, 3, 3, 3, 0, 4, 4, 3, 4, 4, 2, 3, 0, 0, 3, 2, 4, 0, 3, 3, 3, 3, 3, 3, 1, 2, 2, 3, 2, 3, 3, 2, 3]
[4, 0, 4, 2, 1, 3, 3, 4, 4, 3, 3, 3, 0, 2, 0, 3, 0, 2, 4, 3, 3, 3, 1, 2, 2, 3, 4, 3, 0, 1, 1, 3, 1, 3, 3, 2, 4, 0, 3, 3, 0, 0, 4, 4, 2, 0, 3, 3, 3, 2, 2, 3, 4, 1, 2, 3, 3, 2, 0, 3, 2, 4, 4, 2, 1, 0, 1, 2, 3, 2, 3, 4, 2, 1, 4, 2, 3, 3, 3, 1, 3, 3, 3, 3, 2, 3, 3, 3, 4, 3, 3, 3, 3, 4, 4, 4, 3, 3, 1, 4, 3, 1, 4, 3, 3, 3, 3, 4, 1, 4, 2, 3, 0, 4, 4, 3, 3, 1, 2, 3, 1, 3, 3, 1, 3, 1, 3, 3, 3, 3, 2, 3, 3, 3, 2, 3, 2, 4]


...

[7, 1, 11, 13, 11, 11, 11, 11, 15, 11, 11, 11, 9, 11, 15, 11, 11, 13, 11, 11, 1, 13, 9, 11, 13, 11, 3, 15, 11, 9, 11, 13, 6, 11, 4, 11, 13, 11, 9, 11, 11, 11, 11, 11, 11, 11, 11, 11, 7, 15, 11, 2, 2, 7, 14, 14, 5, 11, 7, 11, 11, 11, 7, 5, 7, 11, 11, 11, 4, 7, 11, 11, 11, 7, 11, 11, 11, 2, 11, 13, 7, 7, 3, 11, 11, 7, 11, 7, 11, 9, 11, 11, 11, 11, 11, 6, 11, 13, 11, 14, 11, 11, 11, 11, 11, 11, 11, 7, 4, 11, 11, 11, 11, 13, 11, 11, 11, 13, 4, 11, 3, 11, 7, 11, 11, 7, 4, 11, 11, 11, 0, 11, 11, 11, 10, 7, 11, 11]


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not conve

[1, 9, 11, 11, 6, 11, 11, 14, 11, 11, 11, 11, 11, 11, 11, 11, 11, 13, 11, 11, 11, 10, 6, 11, 9, 7, 9, 11, 7, 6, 11, 9, 11, 11, 11, 9, 11, 11, 7, 11, 11, 2, 7, 11, 4, 11, 11, 11, 11, 14, 15, 15, 13, 11, 11, 11, 12, 15, 7, 6, 3, 11, 14, 11, 2, 6, 11, 11, 11, 7, 11, 6, 11, 6, 11, 6, 11, 11, 11, 11, 13, 3, 7, 3, 4, 11, 15, 6, 11, 11, 11, 11, 0, 13, 7, 11, 11, 14, 13, 11, 11, 4, 11, 11, 11, 12, 7, 13, 13, 7, 11, 7, 15, 8, 3, 13, 4, 11, 2, 11, 11, 11, 7, 11, 11, 7, 9, 11, 3, 2, 11, 11, 11, 11, 7, 10, 5, 11]


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not conve

[7, 11, 7, 11, 10, 2, 14, 11, 11, 11, 11, 11, 11, 2, 13, 13, 3, 11, 11, 13, 14, 11, 11, 13, 11, 11, 4, 7, 11, 6, 11, 11, 11, 9, 15, 11, 11, 11, 9, 7, 11, 11, 11, 11, 5, 3, 11, 15, 11, 11, 11, 15, 11, 11, 11, 7, 7, 11, 11, 11, 11, 7, 11, 7, 9, 11, 11, 11, 11, 3, 11, 7, 3, 11, 7, 3, 11, 11, 3, 7, 4, 11, 15, 11, 11, 11, 11, 4, 4, 11, 11, 11, 6, 11, 11, 13, 11, 11, 13, 11, 13, 11, 11, 11, 11, 11, 11, 11, 13, 11, 11, 11, 11, 11, 13, 4, 11, 12, 6, 13, 11, 11, 11, 11, 11, 11, 15, 11, 11, 11, 11, 11, 4, 11, 11, 11, 11]


In [ ]:
print("Logistic")
print("PERFORMANCE")
print(classification_report(training_data['PERFORMANCE'], training_data['Logistic_PERFORMANCE_Predictions'], zero_division = 1))
print("Label_num_level_1")
print(classification_report(training_data['Label_num_level_1'], training_data['Logistic_Label_num_level_1_Predictions'], zero_division = 1))
print("Label_num_level_2")
print(classification_report(training_data['Label_num_level_2'], training_data['Logistic_Label_num_level_2_Predictions'], zero_division = 1))

Logistic
PERFORMANCE
              precision    recall  f1-score   support

           0       0.83      0.76      0.79       421
           1       0.66      0.75      0.70       268

    accuracy                           0.75       689
   macro avg       0.74      0.75      0.75       689
weighted avg       0.76      0.75      0.76       689

Label_num_level_1
              precision    recall  f1-score   support

           0       0.61      0.70      0.65        43
           1       0.59      0.69      0.64        55
           2       0.63      0.83      0.71        87
           3       0.89      0.74      0.81       421
           4       0.67      0.88      0.76        83

    accuracy                           0.76       689
   macro avg       0.68      0.77      0.71       689
weighted avg       0.79      0.76      0.77       689

Label_num_level_2
              precision    recall  f1-score   support

           0       0.17      0.25      0.20         4
           1      

In [ ]:
k=5
seed = 3
kf = StratifiedKFold(n_splits=k,random_state=seed, shuffle=True)
label_count = {'PERFORMANCE':2,'Label_num_level_1':5,'Label_num_level_2':16}
for label, count in label_count.items():
  x=training_data
  y=training_data[label]
  metrics=[]
  i=1
  for train_index, val_index in kf.split(x,y):
    train_texts, val_texts =x['Concept'].iloc[train_index],x['Concept'].iloc[val_index]
    train_labels, val_labels =y.iloc[train_index],y.iloc[val_index]
    list_text = []
    for words in train_texts:
      list_text.append(words)

    train_embedding = []
    for train in train_texts:
      train_embedding.append(nlp(train))

    train_embedding_final =[]
    for i in train_embedding:
      train_embedding_final.append(i.vector)

    test_embedding = []
    for test in val_texts:
      test_embedding.append(nlp(test))

    test_embedding_final = []
    for j in test_embedding:
      test_embedding_final.append(j.vector)

    param_grid_ = {'C': [1e-5, 1e-3, 1e-1, 1e0, 1e1, 1e2]}
    clf = GridSearchCV(LogisticRegression(solver='saga', max_iter=5000, class_weight='balanced'), cv=5, param_grid=param_grid_).fit(train_embedding_final, train_labels)
    sentence_list = val_texts.to_list()
    prediction_grid = clf.predict(test_embedding_final)
    classification_prediction_list =[]
    for sentence, classification in zip(sentence_list, prediction_grid):
      classification_prediction_list.append(classification)
    print(classification_prediction_list)
    # Naive_Bayes Model metrics
    classification_list = val_labels.to_list()
    indices = val_index
    training_data.loc[indices, "Logistic_{}_Predictions".format(label)]=classification_prediction_list
    #i = i+1

[0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0]
[1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1]
[1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


[4, 3, 2, 3, 1, 4, 3, 0, 4, 0, 3, 2, 3, 4, 3, 3, 4, 3, 3, 4, 3, 4, 4, 3, 2, 3, 3, 3, 2, 4, 3, 3, 3, 4, 4, 2, 3, 0, 2, 2, 4, 4, 0, 4, 3, 0, 3, 1, 3, 4, 3, 3, 3, 4, 2, 3, 3, 3, 3, 1, 3, 2, 2, 3, 3, 3, 1, 1, 2, 3, 2, 0, 4, 0, 4, 3, 3, 3, 0, 3, 4, 2, 4, 3, 3, 3, 3, 3, 2, 3, 3, 3, 3, 3, 3, 2, 3, 3, 3, 4, 4, 2, 3, 4, 4, 3, 3, 1, 1, 3, 2, 1, 1, 2, 0, 4, 0, 4, 3, 0, 4, 0, 1, 0, 2, 1, 3, 3, 3, 2, 3, 3, 4, 3, 2, 3, 2, 4]
[2, 3, 3, 0, 3, 4, 3, 3, 3, 0, 3, 4, 3, 1, 0, 3, 2, 4, 4, 2, 3, 4, 1, 2, 0, 3, 4, 2, 1, 4, 1, 3, 2, 3, 3, 3, 3, 2, 3, 3, 3, 0, 3, 3, 3, 4, 4, 3, 3, 3, 3, 3, 3, 4, 3, 0, 3, 2, 0, 1, 3, 2, 3, 2, 0, 3, 2, 2, 4, 4, 3, 4, 2, 4, 0, 2, 0, 2, 3, 3, 1, 3, 3, 3, 3, 3, 2, 4, 0, 2, 3, 4, 1, 3, 1, 3, 3, 3, 3, 3, 3, 3, 3, 4, 3, 3, 1, 3, 3, 3, 3, 4, 4, 2, 0, 4, 4, 3, 3, 3, 1, 3, 4, 3, 2, 3, 3, 3, 3, 2, 1, 3, 3, 3, 3, 3, 2, 3]


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


[1, 2, 4, 2, 4, 3, 0, 3, 2, 1, 2, 0, 3, 3, 2, 3, 3, 4, 3, 3, 3, 3, 4, 3, 3, 4, 3, 3, 3, 1, 3, 3, 3, 3, 1, 4, 3, 1, 3, 3, 3, 3, 3, 1, 4, 4, 3, 3, 0, 3, 3, 3, 4, 3, 1, 3, 3, 3, 3, 3, 3, 3, 3, 1, 1, 1, 3, 0, 3, 3, 2, 3, 3, 2, 3, 3, 2, 3, 1, 2, 3, 3, 4, 2, 3, 1, 4, 2, 3, 3, 3, 3, 1, 3, 3, 3, 3, 4, 4, 4, 3, 3, 1, 0, 3, 2, 1, 2, 3, 3, 3, 3, 3, 0, 3, 2, 1, 3, 3, 3, 3, 3, 3, 3, 3, 0, 3, 2, 0, 3, 2, 3, 4, 3, 4, 2, 3, 1]


...

[6, 9, 9, 11, 11, 13, 11, 2, 11, 14, 12, 13, 9, 11, 7, 11, 13, 2, 11, 13, 6, 4, 13, 9, 6, 11, 1, 9, 11, 11, 9, 11, 9, 11, 2, 9, 4, 3, 11, 13, 3, 11, 7, 7, 11, 12, 11, 11, 11, 11, 11, 14, 7, 2, 11, 11, 7, 11, 7, 7, 11, 11, 15, 11, 11, 11, 11, 11, 4, 3, 7, 7, 10, 1, 13, 11, 9, 11, 13, 11, 4, 3, 4, 6, 15, 15, 11, 0, 9, 11, 11, 11, 11, 11, 4, 13, 11, 11, 11, 9, 11, 11, 11, 11, 6, 11, 7, 11, 11, 11, 7, 13, 13, 5, 11, 7, 14, 7, 11, 15, 11, 2, 11, 11, 11, 11, 11, 11, 11, 7, 11, 11, 11, 11, 11, 10, 7, 15]


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not conve

[1, 13, 11, 11, 11, 6, 11, 2, 9, 13, 11, 7, 7, 11, 11, 11, 2, 11, 11, 11, 7, 3, 11, 11, 11, 13, 13, 11, 11, 11, 7, 0, 7, 7, 7, 11, 11, 11, 11, 11, 4, 11, 3, 11, 5, 11, 11, 11, 13, 11, 11, 11, 11, 6, 15, 11, 6, 11, 11, 11, 11, 11, 11, 11, 7, 9, 11, 11, 3, 3, 7, 11, 11, 3, 11, 11, 3, 7, 11, 11, 11, 4, 15, 6, 12, 6, 7, 13, 4, 11, 11, 11, 11, 11, 11, 15, 6, 11, 11, 11, 11, 14, 11, 11, 12, 11, 7, 11, 7, 7, 7, 11, 11, 8, 13, 9, 2, 11, 7, 11, 11, 3, 11, 7, 11, 11, 11, 11, 11, 7, 11, 4, 7, 13, 7, 11, 11, 11]


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not conve

[11, 11, 9, 11, 11, 7, 11, 11, 11, 11, 11, 11, 11, 4, 11, 15, 3, 13, 13, 11, 3, 1, 6, 14, 10, 11, 11, 9, 11, 11, 11, 7, 4, 11, 11, 11, 7, 11, 9, 11, 3, 13, 13, 11, 3, 11, 11, 3, 11, 15, 9, 11, 11, 7, 0, 6, 4, 6, 14, 11, 11, 11, 15, 11, 7, 4, 11, 4, 11, 0, 3, 7, 15, 11, 11, 11, 11, 7, 11, 11, 4, 11, 13, 11, 11, 6, 11, 11, 11, 13, 11, 13, 11, 13, 6, 11, 7, 11, 12, 11, 11, 11, 11, 11, 11, 13, 14, 9, 11, 6, 11, 11, 13, 11, 11, 11, 11, 15, 11, 4, 4, 11, 11, 11, 11, 7, 13, 11, 2, 11, 12, 7, 11, 11, 7, 11, 11]


In [ ]:
print("Logistic")
print("PERFORMANCE")
print(classification_report(training_data['PERFORMANCE'], training_data['Logistic_PERFORMANCE_Predictions'], zero_division = 1))
print("Label_num_level_1")
print(classification_report(training_data['Label_num_level_1'], training_data['Logistic_Label_num_level_1_Predictions'], zero_division = 1))
print("Label_num_level_2")
print(classification_report(training_data['Label_num_level_2'], training_data['Logistic_Label_num_level_2_Predictions'], zero_division = 1))

Logistic
PERFORMANCE
              precision    recall  f1-score   support

           0       0.83      0.76      0.79       421
           1       0.67      0.76      0.71       268

    accuracy                           0.76       689
   macro avg       0.75      0.76      0.75       689
weighted avg       0.77      0.76      0.76       689

Label_num_level_1
              precision    recall  f1-score   support

           0       0.61      0.72      0.66        43
           1       0.56      0.67      0.61        55
           2       0.69      0.83      0.75        87
           3       0.89      0.76      0.82       421
           4       0.66      0.86      0.75        83

    accuracy                           0.77       689
   macro avg       0.68      0.77      0.72       689
weighted avg       0.79      0.77      0.78       689

Label_num_level_2
              precision    recall  f1-score   support

           0       0.14      0.25      0.18         4
           1      

In [ ]:
k=5
seed = 4
kf = StratifiedKFold(n_splits=k,random_state=seed, shuffle=True)
label_count = {'PERFORMANCE':2,'Label_num_level_1':5,'Label_num_level_2':16}
for label, count in label_count.items():
  x=training_data
  y=training_data[label]
  metrics=[]
  i=1
  for train_index, val_index in kf.split(x,y):
    train_texts, val_texts =x['Concept'].iloc[train_index],x['Concept'].iloc[val_index]
    train_labels, val_labels =y.iloc[train_index],y.iloc[val_index]
    list_text = []
    for words in train_texts:
      list_text.append(words)

    train_embedding = []
    for train in train_texts:
      train_embedding.append(nlp(train))

    train_embedding_final =[]
    for i in train_embedding:
      train_embedding_final.append(i.vector)

    test_embedding = []
    for test in val_texts:
      test_embedding.append(nlp(test))

    test_embedding_final = []
    for j in test_embedding:
      test_embedding_final.append(j.vector)

    param_grid_ = {'C': [1e-5, 1e-3, 1e-1, 1e0, 1e1, 1e2]}
    clf = GridSearchCV(LogisticRegression(solver='saga', max_iter=5000, class_weight='balanced'), cv=5, param_grid=param_grid_).fit(train_embedding_final, train_labels)
    sentence_list = val_texts.to_list()
    prediction_grid = clf.predict(test_embedding_final)
    classification_prediction_list =[]
    for sentence, classification in zip(sentence_list, prediction_grid):
      classification_prediction_list.append(classification)
    print(classification_prediction_list)
    # Naive_Bayes Model metrics
    classification_list = val_labels.to_list()
    indices = val_index
    training_data.loc[indices, "Logistic_{}_Predictions".format(label)]=classification_prediction_list
    #i = i+1

[1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0]
[0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0]
[1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


[1, 2, 3, 1, 0, 4, 3, 2, 3, 1, 3, 4, 3, 3, 4, 0, 4, 0, 3, 3, 2, 1, 0, 1, 3, 3, 3, 3, 1, 4, 3, 4, 3, 0, 2, 2, 3, 2, 3, 1, 2, 0, 4, 2, 4, 4, 4, 3, 3, 3, 3, 2, 0, 1, 0, 3, 3, 3, 2, 0, 3, 3, 3, 3, 2, 0, 3, 3, 2, 2, 2, 0, 3, 3, 3, 3, 0, 1, 3, 1, 3, 0, 2, 1, 2, 1, 4, 3, 3, 3, 4, 3, 1, 3, 1, 3, 4, 3, 4, 3, 4, 3, 4, 3, 3, 3, 3, 3, 4, 4, 2, 3, 4, 3, 3, 0, 4, 3, 0, 3, 3, 0, 1, 3, 3, 0, 3, 0, 3, 3, 1, 2, 2, 3, 3, 3, 2, 1]


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


[3, 1, 0, 2, 3, 4, 4, 4, 4, 4, 4, 3, 3, 3, 4, 3, 0, 3, 3, 3, 3, 2, 4, 2, 1, 0, 3, 4, 3, 4, 2, 1, 3, 0, 3, 3, 2, 3, 0, 3, 3, 1, 3, 4, 4, 3, 3, 3, 2, 3, 0, 3, 1, 3, 3, 4, 2, 3, 3, 2, 2, 3, 3, 3, 3, 3, 2, 3, 3, 3, 3, 4, 3, 3, 3, 1, 1, 2, 3, 2, 3, 3, 3, 1, 4, 3, 0, 3, 3, 1, 2, 3, 2, 3, 3, 4, 1, 2, 2, 2, 3, 2, 3, 4, 3, 1, 3, 4, 3, 3, 3, 3, 4, 3, 2, 3, 3, 3, 1, 3, 0, 4, 4, 1, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 2, 2]
[1, 4, 3, 3, 4, 3, 3, 0, 3, 2, 2, 3, 0, 3, 4, 4, 2, 2, 4, 2, 3, 2, 3, 3, 2, 3, 1, 0, 3, 3, 3, 4, 3, 4, 0, 3, 3, 2, 1, 1, 1, 3, 0, 3, 4, 3, 2, 0, 3, 3, 2, 3, 2, 0, 2, 3, 3, 3, 3, 1, 3, 3, 2, 2, 3, 1, 2, 3, 3, 2, 3, 3, 3, 4, 3, 3, 0, 2, 3, 4, 3, 4, 1, 3, 1, 1, 2, 3, 3, 3, 3, 4, 3, 3, 3, 2, 3, 4, 3, 4, 3, 3, 4, 4, 1, 3, 3, 3, 3, 3, 2, 4, 4, 3, 0, 2, 3, 3, 3, 4, 0, 4, 1, 0, 1, 3, 3, 1, 2, 3, 1, 3, 3, 3, 4, 2, 3, 4]


...

[5, 9, 11, 15, 11, 7, 9, 11, 10, 11, 11, 11, 11, 11, 11, 11, 11, 2, 1, 6, 11, 11, 11, 13, 7, 11, 11, 11, 11, 13, 1, 9, 11, 11, 9, 11, 7, 11, 11, 11, 2, 14, 11, 5, 11, 3, 13, 13, 11, 12, 2, 6, 15, 11, 11, 11, 11, 11, 14, 7, 5, 11, 3, 11, 11, 3, 6, 5, 11, 7, 11, 11, 7, 7, 7, 11, 9, 7, 7, 3, 11, 11, 15, 3, 11, 6, 11, 11, 11, 4, 11, 9, 11, 11, 11, 11, 11, 15, 11, 11, 13, 11, 11, 11, 11, 4, 11, 6, 4, 11, 11, 11, 11, 13, 14, 11, 15, 11, 8, 13, 11, 11, 2, 11, 11, 7, 11, 11, 11, 11, 11, 11, 4, 12, 7, 11, 15, 11]


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not conve

[14, 11, 11, 13, 3, 11, 2, 14, 6, 13, 4, 15, 11, 0, 9, 11, 3, 4, 11, 11, 13, 11, 9, 11, 11, 11, 11, 13, 11, 7, 11, 5, 3, 11, 11, 3, 11, 11, 11, 7, 13, 13, 15, 15, 11, 7, 11, 11, 3, 6, 15, 15, 14, 11, 11, 11, 2, 11, 6, 6, 11, 11, 11, 11, 7, 2, 11, 11, 11, 14, 6, 11, 11, 11, 11, 9, 4, 7, 11, 11, 11, 2, 4, 7, 3, 11, 4, 7, 11, 11, 11, 11, 13, 11, 11, 6, 11, 13, 6, 11, 11, 13, 11, 13, 6, 11, 11, 11, 11, 11, 12, 13, 7, 11, 11, 7, 7, 11, 11, 15, 7, 11, 11, 3, 7, 11, 11, 11, 11, 7, 7, 11, 7, 11, 10, 11, 11, 7]


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not conve

[13, 11, 11, 11, 14, 11, 9, 11, 11, 11, 11, 11, 3, 11, 11, 11, 11, 9, 6, 15, 11, 14, 11, 11, 11, 11, 11, 11, 6, 9, 11, 11, 11, 11, 13, 11, 11, 11, 7, 11, 2, 11, 7, 11, 11, 11, 11, 11, 7, 11, 11, 7, 9, 4, 4, 7, 11, 11, 6, 15, 11, 11, 7, 3, 11, 3, 11, 11, 4, 11, 7, 2, 11, 11, 13, 13, 7, 7, 11, 11, 11, 11, 11, 11, 15, 7, 11, 7, 0, 9, 11, 11, 13, 11, 11, 11, 11, 11, 11, 11, 11, 11, 13, 14, 13, 7, 11, 11, 11, 11, 13, 13, 9, 1, 4, 7, 9, 11, 3, 15, 3, 11, 2, 11, 7, 3, 7, 11, 11, 2, 10, 11, 11, 11, 11, 7, 9]


In [ ]:
print("Logistic")
print("PERFORMANCE")
print(classification_report(training_data['PERFORMANCE'], training_data['Logistic_PERFORMANCE_Predictions'], zero_division = 1))
print("Label_num_level_1")
print(classification_report(training_data['Label_num_level_1'], training_data['Logistic_Label_num_level_1_Predictions'], zero_division = 1))
print("Label_num_level_2")
print(classification_report(training_data['Label_num_level_2'], training_data['Logistic_Label_num_level_2_Predictions'], zero_division = 1))

Logistic
PERFORMANCE
              precision    recall  f1-score   support

           0       0.82      0.75      0.78       421
           1       0.65      0.74      0.70       268

    accuracy                           0.75       689
   macro avg       0.74      0.75      0.74       689
weighted avg       0.76      0.75      0.75       689

Label_num_level_1
              precision    recall  f1-score   support

           0       0.60      0.79      0.68        43
           1       0.53      0.71      0.60        55
           2       0.65      0.80      0.72        87
           3       0.90      0.74      0.81       421
           4       0.68      0.86      0.76        83

    accuracy                           0.76       689
   macro avg       0.67      0.78      0.72       689
weighted avg       0.79      0.76      0.77       689

Label_num_level_2
              precision    recall  f1-score   support

           0       0.20      0.25      0.22         4
           1      

In [ ]:
train_data = pd.concat([training_data['PERFORMANCE'],training_data['Logistic_PERFORMANCE_Predictions']],axis=1)
train_data_level1 = pd.concat([training_data['Label_num_level_1'], training_data['Logistic_Label_num_level_1_Predictions']],axis=1)
train_data_level2 = pd.concat([training_data['Label_num_level_2'],training_data['Logistic_Label_num_level_2_Predictions']],axis=1)
train_final = pd.concat([train_data,train_data_level1,train_data_level2],axis=1)

In [ ]:
train_final.to_excel('/content/final_spacy.xlsx')

In [ ]:
#BERT embedding

#BERT embedding

In [ ]:
!pip install torch

In [ ]:
!pip install transformers

     |████████████████████████████████| 2.6 MB 7.2 MB/s 
     |████████████████████████████████| 3.3 MB 58.8 MB/s 
     |████████████████████████████████| 895 kB 47.0 MB/s 
     |████████████████████████████████| 636 kB 60.3 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
from transformers import BertTokenizer, BertModel
import pandas as pd
import numpy as np
import nltk
import torch

In [ ]:
# Loading the pre-trained BERT model
###################################
# Embeddings will be derived from
# the outputs of this model
model = BertModel.from_pretrained('bert-base-uncased',
                                  output_hidden_states = True,
                                  )

# Setting up the tokenizer
###################################
# This is the same tokenizer that
# was used in the model to generate
# embeddings to ensure consistency
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')


Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
test = training_data.iloc[:,5]
test1 = training_data.iloc[:,6]
test2 = training_data.iloc[:,8]

In [ ]:
test = pd.concat([test,test1,test2],axis=1)

In [ ]:
test

,PERFORMANCE,Label_num_level_2,Label_num_level_1
0,1,1,0
1,1,7,2
2,1,1,0
3,0,11,3
4,1,4,1
...,...,...,...
684,1,4,1
685,1,7,2
686,0,11,3
687,0,11,3


In [ ]:
!pip install bert-embedding

     |████████████████████████████████| 29.6 MB 36 kB/s 
     |████████████████████████████████| 13.8 MB 22.7 MB/s 
     |████████████████████████████████| 209 kB 67.5 MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.6.0-py3-none-any.whl size=259930 sha256=f1533a20b3a48bbeea17b1473638481ac8f5990c5b361e8d7167c0def56452f1
  Stored in directory: /root/.cache/pip/wheels/a6/41/8f/45bd1c58055d87aee5a71b6756a427ea8d92e506b3a9d17370
Successfully built gluonnlp
  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.5
    Uninstalling numpy-1.19.5:
      Successfully uninstalled numpy-1.19.5
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
xarray 0.18.2 requires numpy>=1.17, but you hav

In [ ]:
from bert_embedding import BertEmbedding
import numpy as np


In [ ]:
k=5
seed = 1
kf = StratifiedKFold(n_splits=k,random_state=seed, shuffle=True)
label_count = {'PERFORMANCE':2,'Label_num_level_1':5,'Label_num_level_2':16}
for label, count in label_count.items():
  x=training_data
  y=training_data[label]
  metrics=[]
  i=1
  for train_index, val_index in kf.split(x,y):
    train_texts, val_texts =x['Concept'].iloc[train_index],x['Concept'].iloc[val_index]
    train_labels, val_labels =y.iloc[train_index],y.iloc[val_index]
    list_text = []
    for words in train_texts:
      list_text.append(words)

    result_embeddings = []
    bert_embedding = BertEmbedding()
    for i in train_texts:
      sentences = i.split('\n')
      result = bert_embedding(sentences)
      result_embeddings.append(result)

    result_test_embeddings = []
    bert_embedding = BertEmbedding()
    for i in val_texts:
      sentences = i.split('\n')
      result = bert_embedding(sentences)
      result_test_embeddings.append(result)

    # result_embeddings_new = np.asarray(result_embeddings)
    # result_embeddings_test_new = np.asarray(result_test_embeddings)
    # s= (np.shape(result_embeddings_new))[0]
    # t= (np.shape(result_embeddings_new))[2]
    # print(s)
    # print(t)
    # print(np.shape(result_embeddings_new))
    # print(np.shape(result_embeddings_test_new))
    # result_embeddings_new = result_embeddings_new.reshape()
    # result_embeddings_test_new = result_embeddings_test_new.reshape()

    result = []
    for i in result_embeddings:
      for j in i:
        #print(j[1])
        result.append(j[1])

    result_test = []
    for i in result_test_embeddings:
      for j in i:
        result_test.append(j[1])

    k = 0
    result_new1 = []
    for i in result:
      h = np.shape(i)
      avg = h[0]
      for j in i:
        k = (k + j)/avg
      result_new1.append(k)

    k = 0
    result_test_new1 = []
    for i in result_test:
      h = np.shape(i)
      avg = h[0]
      for j in i:
        k = (k + j)/avg
      result_test_new1.append(k)

    param_grid_ = {'C': [1e-5, 1e-3, 1e-1, 1e0, 1e1, 1e2]}
    clf = GridSearchCV(LogisticRegression(solver='saga', max_iter=5000, class_weight='balanced'), cv=5, param_grid=param_grid_).fit(result_new1, train_labels)
    sentence_list = val_texts.to_list()
    prediction_grid = clf.predict(result_test_new1)
    classification_prediction_list =[]
    for sentence, classification in zip(sentence_list, prediction_grid):
      classification_prediction_list.append(classification)
    print(classification_prediction_list)
    # Naive_Bayes Model metrics
    classification_list = val_labels.to_list()
    indices = val_index
    training_data.loc[indices, "Logistic_{}_Predictions".format(label)]=classification_prediction_list

Vocab file is not found. Downloading.


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


[1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1]
[1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0]
[1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


[3, 3, 4, 2, 3, 3, 3, 3, 2, 3, 3, 3, 0, 3, 4, 3, 3, 4, 3, 3, 3, 3, 4, 2, 4, 2, 4, 3, 1, 3, 4, 2, 3, 2, 4, 3, 3, 3, 3, 0, 3, 2, 1, 3, 3, 3, 2, 3, 4, 3, 3, 4, 3, 3, 3, 3, 3, 4, 3, 3, 4, 3, 3, 3, 3, 1, 1, 4, 2, 3, 3, 3, 3, 3, 3, 1, 3, 3, 3, 3, 0, 1, 3, 3, 3, 4, 1, 4, 3, 3, 1, 4, 3, 3, 3, 4, 4, 3, 3, 4, 1, 3, 4, 2, 3, 4, 4, 4, 3, 3, 2, 1, 3, 3, 3, 3, 4, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 2, 3, 3, 2, 3, 2]
[2, 2, 3, 3, 0, 0, 3, 4, 2, 3, 4, 3, 0, 4, 4, 4, 3, 3, 3, 3, 3, 3, 1, 3, 0, 3, 2, 1, 3, 1, 3, 4, 1, 0, 2, 3, 3, 1, 3, 4, 3, 4, 3, 4, 3, 0, 3, 3, 0, 3, 3, 3, 3, 3, 3, 1, 2, 3, 3, 3, 2, 3, 3, 3, 3, 2, 3, 4, 3, 1, 3, 0, 3, 2, 2, 3, 1, 3, 3, 3, 3, 3, 3, 3, 3, 3, 2, 2, 1, 1, 4, 3, 2, 3, 4, 3, 1, 1, 3, 3, 4, 3, 3, 2, 3, 3, 4, 3, 3, 1, 4, 3, 4, 1, 2, 4, 4, 3, 3, 1, 3, 1, 3, 3, 0, 2, 3, 2, 3, 3, 3, 0, 2, 2, 3, 3, 3]


...

[7, 1, 11, 13, 11, 11, 9, 11, 11, 11, 11, 11, 11, 11, 5, 11, 11, 13, 11, 11, 1, 13, 9, 11, 14, 9, 14, 11, 11, 9, 9, 13, 6, 11, 11, 11, 13, 11, 9, 11, 11, 11, 11, 7, 15, 11, 11, 11, 7, 15, 11, 2, 2, 11, 3, 7, 5, 11, 11, 11, 7, 2, 2, 11, 4, 11, 3, 3, 4, 7, 11, 11, 11, 7, 11, 11, 2, 2, 11, 13, 7, 7, 11, 11, 11, 11, 11, 7, 11, 9, 11, 11, 11, 11, 11, 4, 11, 13, 11, 13, 4, 11, 11, 11, 11, 11, 11, 7, 4, 9, 11, 11, 11, 13, 11, 11, 11, 13, 4, 11, 7, 11, 11, 11, 11, 7, 4, 11, 11, 11, 11, 11, 11, 7, 7, 11, 11, 11]


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)


[1, 11, 11, 11, 11, 11, 11, 14, 11, 15, 11, 11, 11, 11, 11, 11, 11, 13, 11, 11, 11, 10, 6, 11, 2, 7, 9, 11, 11, 6, 7, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 7, 11, 11, 11, 11, 11, 11, 11, 11, 11, 13, 11, 11, 11, 12, 11, 4, 6, 3, 11, 14, 11, 11, 6, 11, 11, 11, 4, 11, 9, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 3, 4, 11, 15, 6, 11, 11, 11, 11, 11, 13, 7, 11, 11, 13, 13, 13, 11, 4, 11, 11, 14, 12, 7, 13, 13, 7, 11, 7, 15, 7, 11, 13, 4, 11, 3, 11, 3, 11, 11, 11, 11, 7, 11, 11, 11, 2, 11, 11, 11, 11, 7, 10, 5, 13]


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


[11, 11, 7, 11, 10, 10, 11, 11, 6, 11, 11, 11, 11, 11, 11, 11, 14, 11, 11, 13, 14, 9, 11, 11, 11, 11, 4, 7, 11, 6, 11, 11, 2, 7, 11, 11, 11, 7, 9, 7, 11, 11, 11, 11, 5, 13, 11, 11, 11, 11, 11, 15, 11, 11, 11, 11, 11, 7, 11, 11, 7, 11, 11, 7, 11, 11, 11, 11, 11, 3, 11, 9, 3, 3, 11, 3, 11, 11, 11, 7, 4, 4, 4, 11, 11, 11, 11, 4, 4, 11, 11, 11, 11, 11, 6, 13, 11, 11, 11, 11, 13, 13, 11, 11, 11, 7, 3, 11, 14, 11, 11, 11, 11, 11, 13, 11, 11, 12, 11, 13, 11, 11, 11, 11, 11, 11, 15, 11, 11, 11, 11, 11, 4, 7, 11, 11, 11]


In [ ]:
print("Logistic")
print("PERFORMANCE")
print(classification_report(training_data['PERFORMANCE'], training_data['Logistic_PERFORMANCE_Predictions'], zero_division = 1))
print("Label_num_level_1")
print(classification_report(training_data['Label_num_level_1'], training_data['Logistic_Label_num_level_1_Predictions'], zero_division = 1))
print("Label_num_level_2")
print(classification_report(training_data['Label_num_level_2'], training_data['Logistic_Label_num_level_2_Predictions'], zero_division = 1))

Logistic
PERFORMANCE
              precision    recall  f1-score   support

           0       0.79      0.80      0.79       421
           1       0.68      0.68      0.68       268

    accuracy                           0.75       689
   macro avg       0.74      0.74      0.74       689
weighted avg       0.75      0.75      0.75       689

Label_num_level_1
              precision    recall  f1-score   support

           0       0.49      0.47      0.48        43
           1       0.54      0.58      0.56        55
           2       0.63      0.67      0.65        87
           3       0.83      0.79      0.81       421
           4       0.67      0.76      0.71        83

    accuracy                           0.74       689
   macro avg       0.63      0.65      0.64       689
weighted avg       0.74      0.74      0.74       689

Label_num_level_2
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         4
           1      

In [ ]:
k=5
seed = 2
kf = StratifiedKFold(n_splits=k,random_state=seed, shuffle=True)
label_count = {'PERFORMANCE':2,'Label_num_level_1':5,'Label_num_level_2':16}
for label, count in label_count.items():
  x=training_data
  y=training_data[label]
  metrics=[]
  i=1
  for train_index, val_index in kf.split(x,y):
    train_texts, val_texts =x['Concept'].iloc[train_index],x['Concept'].iloc[val_index]
    train_labels, val_labels =y.iloc[train_index],y.iloc[val_index]
    list_text = []
    for words in train_texts:
      list_text.append(words)

    result_embeddings = []
    bert_embedding = BertEmbedding()
    for i in train_texts:
      sentences = i.split('\n')
      result = bert_embedding(sentences)
      result_embeddings.append(result)

    result_test_embeddings = []
    bert_embedding = BertEmbedding()
    for i in val_texts:
      sentences = i.split('\n')
      result = bert_embedding(sentences)
      result_test_embeddings.append(result)

    # result_embeddings_new = np.asarray(result_embeddings)
    # result_embeddings_test_new = np.asarray(result_test_embeddings)
    # s= (np.shape(result_embeddings_new))[0]
    # t= (np.shape(result_embeddings_new))[2]
    # print(s)
    # print(t)
    # print(np.shape(result_embeddings_new))
    # print(np.shape(result_embeddings_test_new))
    # result_embeddings_new = result_embeddings_new.reshape()
    # result_embeddings_test_new = result_embeddings_test_new.reshape()

    result = []
    for i in result_embeddings:
      for j in i:
        #print(j[1])
        result.append(j[1])

    result_test = []
    for i in result_test_embeddings:
      for j in i:
        result_test.append(j[1])

    k = 0
    result_new1 = []
    for i in result:
      h = np.shape(i)
      avg = h[0]
      for j in i:
        k = (k + j)/avg
      result_new1.append(k)

    k = 0
    result_test_new1 = []
    for i in result_test:
      h = np.shape(i)
      avg = h[0]
      for j in i:
        k = (k + j)/avg
      result_test_new1.append(k)

    param_grid_ = {'C': [1e-5, 1e-3, 1e-1, 1e0, 1e1, 1e2]}
    clf = GridSearchCV(LogisticRegression(solver='saga', max_iter=5000, class_weight='balanced'), cv=5, param_grid=param_grid_).fit(result_new1, train_labels)
    sentence_list = val_texts.to_list()
    prediction_grid = clf.predict(result_test_new1)
    classification_prediction_list =[]
    for sentence, classification in zip(sentence_list, prediction_grid):
      classification_prediction_list.append(classification)
    print(classification_prediction_list)
    # Naive_Bayes Model metrics
    classification_list = val_labels.to_list()
    indices = val_index
    training_data.loc[indices, "Logistic_{}_Predictions".format(label)]=classification_prediction_list

Vocab file is not found. Downloading.
[1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1]
[1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1]
[1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0,

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


[1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1]
[3, 2, 3, 3, 3, 3, 1, 0, 0, 4, 3, 2, 3, 4, 3, 4, 3, 3, 0, 2, 3, 3, 4, 3, 3, 1, 3, 3, 2, 1, 3, 2, 2, 0, 3, 2, 3, 3, 2, 3, 3, 4, 3, 3, 4, 3, 3, 3, 3, 3, 3, 1, 4, 0, 3, 3, 3, 0, 3, 4, 1, 3, 3, 2, 0, 3, 3, 3, 4, 3, 3, 1, 2, 3, 3, 3, 3, 1, 3, 2, 3, 3, 3, 2, 4, 4, 1, 2, 2, 2, 1, 3, 3, 1, 2, 3, 3, 3, 3, 4, 4, 3, 3, 3, 4, 4, 3, 3, 3, 3, 2, 4, 3, 1, 3, 3, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 2, 2, 3, 3, 3, 2, 2, 3, 3, 3, 4]
[0, 4, 3, 0, 0, 2, 3, 3, 3, 4, 3, 4, 4, 3, 3, 3, 3, 3, 0, 4, 3, 3, 4, 3, 3, 3, 3, 3, 4, 2, 3, 0, 3, 4, 0, 1, 1, 3, 3, 3, 2, 2, 3, 3, 4, 3, 4, 3, 3, 3, 3, 3, 1, 2, 3, 3, 1, 3

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)


[14, 11, 11, 14, 11, 11, 11, 11, 4, 11, 13, 13, 11, 14, 11, 14, 10, 6, 11, 11, 11, 13, 9, 14, 7, 11, 11, 11, 1, 9, 11, 11, 11, 11, 11, 7, 11, 11, 9, 9, 11, 7, 11, 3, 11, 5, 6, 11, 11, 13, 7, 3, 11, 11, 11, 2, 6, 11, 7, 11, 11, 11, 5, 2, 11, 11, 11, 7, 7, 11, 11, 11, 11, 11, 11, 11, 11, 11, 7, 11, 11, 11, 4, 15, 11, 11, 11, 11, 11, 7, 11, 15, 4, 11, 11, 11, 14, 11, 11, 7, 7, 4, 11, 11, 11, 11, 7, 11, 13, 11, 11, 7, 11, 6, 13, 11, 4, 11, 11, 11, 11, 11, 11, 11, 13, 13, 2, 7, 7, 11, 11, 7, 7, 11, 11, 11, 11, 5]


...

[1, 11, 11, 11, 9, 13, 11, 9, 11, 11, 13, 11, 10, 11, 6, 11, 11, 13, 11, 11, 11, 11, 11, 2, 11, 9, 11, 7, 6, 11, 7, 11, 7, 11, 11, 11, 11, 4, 7, 4, 6, 6, 11, 5, 13, 11, 7, 11, 11, 11, 15, 12, 2, 11, 11, 11, 3, 11, 11, 10, 11, 4, 4, 7, 11, 15, 11, 11, 6, 11, 3, 11, 11, 11, 15, 15, 11, 11, 11, 13, 7, 15, 9, 7, 15, 11, 11, 11, 13, 11, 11, 11, 11, 11, 11, 11, 13, 11, 13, 9, 11, 11, 7, 11, 3, 13, 12, 7, 11, 11, 14, 7, 11, 11, 11, 4, 11, 11, 7, 11, 11, 11, 11, 11, 3, 11, 11, 11, 11, 11, 11, 11, 7, 7, 11, 7, 7, 13]


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)


[7, 1, 15, 2, 13, 11, 15, 11, 11, 9, 11, 14, 6, 11, 7, 2, 11, 9, 15, 6, 7, 11, 11, 11, 2, 13, 11, 7, 11, 11, 2, 9, 11, 11, 13, 11, 11, 11, 11, 11, 7, 11, 11, 11, 3, 11, 11, 7, 6, 6, 11, 13, 11, 11, 2, 11, 11, 3, 4, 7, 7, 11, 14, 11, 11, 11, 11, 11, 9, 11, 11, 4, 11, 11, 11, 4, 11, 7, 7, 11, 7, 13, 9, 11, 13, 13, 11, 11, 11, 11, 6, 11, 11, 11, 11, 13, 6, 7, 11, 7, 11, 11, 13, 11, 11, 11, 11, 11, 11, 3, 11, 7, 13, 13, 11, 3, 11, 15, 11, 7, 4, 4, 11, 11, 14, 11, 13, 7, 11, 11, 11, 11, 11, 7, 11, 11, 7, 11]


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


[11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 9, 2, 7, 11, 11, 11, 11, 7, 14, 11, 11, 13, 11, 11, 11, 11, 7, 11, 13, 11, 11, 11, 11, 4, 7, 11, 11, 11, 14, 11, 13, 11, 11, 13, 11, 11, 11, 2, 11, 7, 15, 14, 11, 11, 11, 11, 11, 7, 11, 11, 11, 11, 7, 11, 11, 6, 11, 11, 11, 3, 3, 11, 4, 3, 11, 11, 7, 4, 11, 11, 11, 11, 7, 2, 11, 11, 11, 4, 11, 11, 15, 11, 11, 11, 7, 6, 3, 11, 11, 11, 4, 11, 13, 15, 13, 7, 13, 11, 11, 11, 11, 11, 11, 11, 11, 11, 12, 11, 11, 13, 11, 11, 11, 15, 11, 11, 11, 13, 11, 11, 11, 11, 11, 11, 7, 11, 11]


In [ ]:
print("Logistic")
print("PERFORMANCE")
print(classification_report(training_data['PERFORMANCE'], training_data['Logistic_PERFORMANCE_Predictions'], zero_division = 1))
print("Label_num_level_1")
print(classification_report(training_data['Label_num_level_1'], training_data['Logistic_Label_num_level_1_Predictions'], zero_division = 1))
print("Label_num_level_2")
print(classification_report(training_data['Label_num_level_2'], training_data['Logistic_Label_num_level_2_Predictions'], zero_division = 1))

Logistic
PERFORMANCE
              precision    recall  f1-score   support

           0       0.81      0.81      0.81       421
           1       0.69      0.69      0.69       268

    accuracy                           0.76       689
   macro avg       0.75      0.75      0.75       689
weighted avg       0.76      0.76      0.76       689

Label_num_level_1
              precision    recall  f1-score   support

           0       0.58      0.58      0.58        43
           1       0.57      0.64      0.60        55
           2       0.62      0.64      0.63        87
           3       0.82      0.79      0.81       421
           4       0.70      0.75      0.72        83

    accuracy                           0.74       689
   macro avg       0.66      0.68      0.67       689
weighted avg       0.75      0.74      0.74       689

Label_num_level_2
              precision    recall  f1-score   support

           0       1.00      0.00      0.00         4
           1      

In [ ]:
k=5
seed = 3
kf = StratifiedKFold(n_splits=k,random_state=seed, shuffle=True)
label_count = {'PERFORMANCE':2,'Label_num_level_1':5,'Label_num_level_2':16}
for label, count in label_count.items():
  x=training_data
  y=training_data[label]
  metrics=[]
  i=1
  for train_index, val_index in kf.split(x,y):
    train_texts, val_texts =x['Concept'].iloc[train_index],x['Concept'].iloc[val_index]
    train_labels, val_labels =y.iloc[train_index],y.iloc[val_index]
    list_text = []
    for words in train_texts:
      list_text.append(words)

    result_embeddings = []
    bert_embedding = BertEmbedding()
    for i in train_texts:
      sentences = i.split('\n')
      result = bert_embedding(sentences)
      result_embeddings.append(result)

    result_test_embeddings = []
    bert_embedding = BertEmbedding()
    for i in val_texts:
      sentences = i.split('\n')
      result = bert_embedding(sentences)
      result_test_embeddings.append(result)

    # result_embeddings_new = np.asarray(result_embeddings)
    # result_embeddings_test_new = np.asarray(result_test_embeddings)
    # s= (np.shape(result_embeddings_new))[0]
    # t= (np.shape(result_embeddings_new))[2]
    # print(s)
    # print(t)
    # print(np.shape(result_embeddings_new))
    # print(np.shape(result_embeddings_test_new))
    # result_embeddings_new = result_embeddings_new.reshape()
    # result_embeddings_test_new = result_embeddings_test_new.reshape()

    result = []
    for i in result_embeddings:
      for j in i:
        #print(j[1])
        result.append(j[1])

    result_test = []
    for i in result_test_embeddings:
      for j in i:
        result_test.append(j[1])

    k = 0
    result_new1 = []
    for i in result:
      h = np.shape(i)
      avg = h[0]
      for j in i:
        k = (k + j)/avg
      result_new1.append(k)

    k = 0
    result_test_new1 = []
    for i in result_test:
      h = np.shape(i)
      avg = h[0]
      for j in i:
        k = (k + j)/avg
      result_test_new1.append(k)

    param_grid_ = {'C': [1e-5, 1e-3, 1e-1, 1e0, 1e1, 1e2]}
    clf = GridSearchCV(LogisticRegression(solver='saga', max_iter=5000, class_weight='balanced'), cv=5, param_grid=param_grid_).fit(result_new1, train_labels)
    sentence_list = val_texts.to_list()
    prediction_grid = clf.predict(result_test_new1)
    classification_prediction_list =[]
    for sentence, classification in zip(sentence_list, prediction_grid):
      classification_prediction_list.append(classification)
    print(classification_prediction_list)
    # Naive_Bayes Model metrics
    classification_list = val_labels.to_list()
    indices = val_index
    training_data.loc[indices, "Logistic_{}_Predictions".format(label)]=classification_prediction_list

Vocab file is not found. Downloading.
[0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0]
[1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0]
[1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0,

In [ ]:
print("Logistic")
print("PERFORMANCE")
print(classification_report(training_data['PERFORMANCE'], training_data['Logistic_PERFORMANCE_Predictions'], zero_division = 1))
print("Label_num_level_1")
print(classification_report(training_data['Label_num_level_1'], training_data['Logistic_Label_num_level_1_Predictions'], zero_division = 1))
print("Label_num_level_2")
print(classification_report(training_data['Label_num_level_2'], training_data['Logistic_Label_num_level_2_Predictions'], zero_division = 1))

In [ ]:
k=5
seed = 4
kf = StratifiedKFold(n_splits=k,random_state=seed, shuffle=True)
label_count = {'PERFORMANCE':2,'Label_num_level_1':5,'Label_num_level_2':16}
for label, count in label_count.items():
  x=training_data
  y=training_data[label]
  metrics=[]
  i=1
  for train_index, val_index in kf.split(x,y):
    train_texts, val_texts =x['Concept'].iloc[train_index],x['Concept'].iloc[val_index]
    train_labels, val_labels =y.iloc[train_index],y.iloc[val_index]
    list_text = []
    for words in train_texts:
      list_text.append(words)

    result_embeddings = []
    bert_embedding = BertEmbedding()
    for i in train_texts:
      sentences = i.split('\n')
      result = bert_embedding(sentences)
      result_embeddings.append(result)

    result_test_embeddings = []
    bert_embedding = BertEmbedding()
    for i in val_texts:
      sentences = i.split('\n')
      result = bert_embedding(sentences)
      result_test_embeddings.append(result)

    # result_embeddings_new = np.asarray(result_embeddings)
    # result_embeddings_test_new = np.asarray(result_test_embeddings)
    # s= (np.shape(result_embeddings_new))[0]
    # t= (np.shape(result_embeddings_new))[2]
    # print(s)
    # print(t)
    # print(np.shape(result_embeddings_new))
    # print(np.shape(result_embeddings_test_new))
    # result_embeddings_new = result_embeddings_new.reshape()
    # result_embeddings_test_new = result_embeddings_test_new.reshape()

    result = []
    for i in result_embeddings:
      for j in i:
        #print(j[1])
        result.append(j[1])

    result_test = []
    for i in result_test_embeddings:
      for j in i:
        result_test.append(j[1])

    k = 0
    result_new1 = []
    for i in result:
      h = np.shape(i)
      avg = h[0]
      for j in i:
        k = (k + j)/avg
      result_new1.append(k)

    k = 0
    result_test_new1 = []
    for i in result_test:
      h = np.shape(i)
      avg = h[0]
      for j in i:
        k = (k + j)/avg
      result_test_new1.append(k)

    param_grid_ = {'C': [1e-5, 1e-3, 1e-1, 1e0, 1e1, 1e2]}
    clf = GridSearchCV(LogisticRegression(solver='saga', max_iter=5000, class_weight='balanced'), cv=5, param_grid=param_grid_).fit(result_new1, train_labels)
    sentence_list = val_texts.to_list()
    prediction_grid = clf.predict(result_test_new1)
    classification_prediction_list =[]
    for sentence, classification in zip(sentence_list, prediction_grid):
      classification_prediction_list.append(classification)
    print(classification_prediction_list)
    # Naive_Bayes Model metrics
    classification_list = val_labels.to_list()
    indices = val_index
    training_data.loc[indices, "Logistic_{}_Predictions".format(label)]=classification_prediction_list

[0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0]
[0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0]
[1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0

In [ ]:
print("Logistic")
print("PERFORMANCE")
print(classification_report(training_data['PERFORMANCE'], training_data['Logistic_PERFORMANCE_Predictions'], zero_division = 1))
print("Label_num_level_1")
print(classification_report(training_data['Label_num_level_1'], training_data['Logistic_Label_num_level_1_Predictions'], zero_division = 1))
print("Label_num_level_2")
print(classification_report(training_data['Label_num_level_2'], training_data['Logistic_Label_num_level_2_Predictions'], zero_division = 1))

In [ ]:
#Fine tuning BERT

In [ ]:
!pip install pandas transformers

     |████████████████████████████████| 2.6 MB 8.4 MB/s 
     |████████████████████████████████| 636 kB 70.6 MB/s 
     |████████████████████████████████| 895 kB 76.3 MB/s 
     |████████████████████████████████| 3.3 MB 61.6 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
from pandas import read_csv
import pandas as pd
import math
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_recall_fscore_support
import torch
from transformers import Trainer, TrainingArguments
from transformers import BertTokenizerFast
from transformers import BertForSequenceClassification, BertConfig

In [ ]:
# Method to get level_1 labels from level_2 labels
max_length=0
def label_types (row):
  global max_length
  length = len(row['Label'])
  max_length = max(max_length,length)
  if length == 4 :
    return row['Label'][0:3]
  else: return row['Label']

In [ ]:
# training_data = read_csv('organizational-performance-dataset.csv')
# training_data['Concept'] = training_data.apply (lambda row: lemmatizeSentence(row), axis=1)
# training_data['Label'] = pd.Categorical(training_data['Label'])
# training_data['Label_num_level_2']=-1
# # training_data['Label_num_level_2'] = training_data['Label'].cat.codes
# # training_data['Label_num_level_2'].astype('int64')
# training_data['Label_level_1'] = training_data.apply (lambda row: label_types(row), axis=1)
# training_data['Label_level_1'] = pd.Categorical(training_data['Label_level_1'])
# training_data['Label_num_level_1'] = training_data['Label_level_1'].cat.codes
# training_data['Label_num_level_1'] = -1
# training_data=training_data.apply (covertLabelsToNum, axis=1)
# training_data.head(20)

In [ ]:
#training_data.Label_level_1.unique().to_list()

In [ ]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted',zero_division=0)
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

class OrgPerformanceDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

model_name = "bert-base-uncased"
tokenizer = BertTokenizerFast.from_pretrained(model_name)

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
from transformers import DistilBertTokenizerFast
from transformers import DistilBertForSequenceClassification, Trainer, TrainingArguments, DistilBertConfig
class OrgPerDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted',zero_division=0)
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

In [ ]:
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')
label_count = {'PERFORMANCE':2,'Label_num_level_1':5,'Label_num_level_2':16}
k=5
seed = 1
kf = StratifiedKFold(n_splits=k,random_state=seed, shuffle=True)
for label, count in label_count.items():
  x=training_data['Concept']
  y=training_data[label]
  metrics=[]
  i=0
  for train_index, val_index in kf.split(x,y):
    train_texts, val_texts =x.iloc[train_index].tolist(),x.iloc[val_index].tolist()
    train_labels, val_labels =y.iloc[train_index].tolist(),y.iloc[val_index].tolist()

    train_encodings = tokenizer(train_texts, truncation=True, padding=True)
    val_encodings = tokenizer(val_texts, truncation=True, padding=True)


    train_dataset = OrgPerDataset(train_encodings, train_labels)
    val_dataset = OrgPerDataset(val_encodings, val_labels)

    training_args = TrainingArguments(
        output_dir='./results',          # output directory
        num_train_epochs=3,              # total number of training epochs
        per_device_train_batch_size=16,  # batch size per device during training
        per_device_eval_batch_size=64,   # batch size for evaluation
        warmup_steps=500,                # number of warmup steps for learning rate scheduler
        weight_decay=0.01,               # strength of weight decay
        logging_dir='./logs',            # directory for storing logs
        logging_steps=10,
    )
    config = DistilBertConfig.from_pretrained('distilbert-base-uncased')
    config.num_labels = count
    model = DistilBertForSequenceClassification(config)

    trainer = Trainer(
        model=model,                         # the instantiated 🤗 Transformers model to be trained
        args=training_args,                  # training arguments, defined above
        train_dataset=train_dataset,         # training dataset
        eval_dataset=val_dataset             # evaluation dataset
    )

    trainer.train()
    pred = trainer.predict(val_dataset)
    preds = pred.predictions.argmax(-1)
    indices = val_index
    training_data.loc[indices, "BERT_{}_Predictions".format(label)]=pd.Series(preds).tolist()

https://huggingface.co/distilbert-base-uncased/resolve/main/vocab.txt not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmpkp00qwfq


Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

storing https://huggingface.co/distilbert-base-uncased/resolve/main/vocab.txt in cache at /root/.cache/huggingface/transformers/0e1bbfda7f63a99bb52e3915dcf10c3c92122b827d92eb2d34ce94ee79ba486c.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99
creating metadata file for /root/.cache/huggingface/transformers/0e1bbfda7f63a99bb52e3915dcf10c3c92122b827d92eb2d34ce94ee79ba486c.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99
https://huggingface.co/distilbert-base-uncased/resolve/main/tokenizer.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmp386_eufo


Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

storing https://huggingface.co/distilbert-base-uncased/resolve/main/tokenizer.json in cache at /root/.cache/huggingface/transformers/75abb59d7a06f4f640158a9bfcde005264e59e8d566781ab1415b139d2e4c603.7f2721073f19841be16f41b0a70b600ca6b880c8f3df6f3535cbc704371bdfa4
creating metadata file for /root/.cache/huggingface/transformers/75abb59d7a06f4f640158a9bfcde005264e59e8d566781ab1415b139d2e4c603.7f2721073f19841be16f41b0a70b600ca6b880c8f3df6f3535cbc704371bdfa4
https://huggingface.co/distilbert-base-uncased/resolve/main/tokenizer_config.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmpg_jm78xm


...

Step,Training Loss
10,2.705800
20,2.518300
30,2.202000
40,1.765800
50,1.877500
60,1.578800
70,1.677400
80,1.624100
90,1.590500
100,1.626700




Training completed. Do not forget to share your model on huggingface.co/models =)


***** Running Prediction *****
  Num examples = 138
  Batch size = 64


PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
loading configuration file https://huggingface.co/distilbert-base-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/23454919702d26495337f3da04d1655c7ee010d5ec9d77bdb9e399e00302c0a1.d423bdf2f58dc8b77d5f5d18028d7ae4a72dcfd8f468e81fe979ada957a8c361
Model config DistilBertConfig {
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,

Step,Training Loss
10,2.678700
20,2.557600
30,2.216800
40,1.897100
50,1.796700
60,1.587100
70,1.751900
80,1.441500
90,1.680500
100,1.649900




Training completed. Do not forget to share your model on huggingface.co/models =)


***** Running Prediction *****
  Num examples = 137
  Batch size = 64


In [ ]:
# Performance BERT Metrics
print(classification_report(training_data['PERFORMANCE'], training_data['BERT_PERFORMANCE_Predictions'], zero_division = 1))
# Label_num_level_1 BERT Metrics
print(classification_report(training_data['Label_num_level_1'], training_data['BERT_Label_num_level_1_Predictions'], zero_division = 1))
#Label_num_level_2 BERT Metrics
print(classification_report(training_data['Label_num_level_2'], training_data['BERT_Label_num_level_2_Predictions'], zero_division = 1))

              precision    recall  f1-score   support

           0       0.62      0.99      0.76       421
           1       0.64      0.03      0.05       268

    accuracy                           0.62       689
   macro avg       0.63      0.51      0.40       689
weighted avg       0.62      0.62      0.48       689

              precision    recall  f1-score   support

           0       1.00      0.00      0.00        43
           1       1.00      0.00      0.00        55
           2       1.00      0.00      0.00        87
           3       0.61      1.00      0.76       421
           4       1.00      0.00      0.00        83

    accuracy                           0.61       689
   macro avg       0.92      0.20      0.15       689
weighted avg       0.76      0.61      0.46       689

              precision    recall  f1-score   support

           0       1.00      0.00      0.00         4
           1       1.00      0.00      0.00         4
           2       1.

In [ ]:
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')
label_count = {'PERFORMANCE':2,'Label_num_level_1':5,'Label_num_level_2':16}
k=5
seed = 2
kf = StratifiedKFold(n_splits=k,random_state=seed, shuffle=True)
for label, count in label_count.items():
  x=training_data['Concept']
  y=training_data[label]
  metrics=[]
  i=0
  for train_index, val_index in kf.split(x,y):
    train_texts, val_texts =x.iloc[train_index].tolist(),x.iloc[val_index].tolist()
    train_labels, val_labels =y.iloc[train_index].tolist(),y.iloc[val_index].tolist()

    train_encodings = tokenizer(train_texts, truncation=True, padding=True)
    val_encodings = tokenizer(val_texts, truncation=True, padding=True)


    train_dataset = OrgPerDataset(train_encodings, train_labels)
    val_dataset = OrgPerDataset(val_encodings, val_labels)

    training_args = TrainingArguments(
        output_dir='./results',          # output directory
        num_train_epochs=3,              # total number of training epochs
        per_device_train_batch_size=16,  # batch size per device during training
        per_device_eval_batch_size=64,   # batch size for evaluation
        warmup_steps=500,                # number of warmup steps for learning rate scheduler
        weight_decay=0.01,               # strength of weight decay
        logging_dir='./logs',            # directory for storing logs
        logging_steps=10,
    )
    config = DistilBertConfig.from_pretrained('distilbert-base-uncased')
    config.num_labels = count
    model = DistilBertForSequenceClassification(config)

    trainer = Trainer(
        model=model,                         # the instantiated 🤗 Transformers model to be trained
        args=training_args,                  # training arguments, defined above
        train_dataset=train_dataset,         # training dataset
        eval_dataset=val_dataset             # evaluation dataset
    )

    trainer.train()
    pred = trainer.predict(val_dataset)
    preds = pred.predictions.argmax(-1)
    indices = val_index
    training_data.loc[indices, "BERT_{}_Predictions".format(label)]=pd.Series(preds).tolist()

loading file https://huggingface.co/distilbert-base-uncased/resolve/main/vocab.txt from cache at /root/.cache/huggingface/transformers/0e1bbfda7f63a99bb52e3915dcf10c3c92122b827d92eb2d34ce94ee79ba486c.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99
loading file https://huggingface.co/distilbert-base-uncased/resolve/main/tokenizer.json from cache at /root/.cache/huggingface/transformers/75abb59d7a06f4f640158a9bfcde005264e59e8d566781ab1415b139d2e4c603.7f2721073f19841be16f41b0a70b600ca6b880c8f3df6f3535cbc704371bdfa4
loading file https://huggingface.co/distilbert-base-uncased/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/distilbert-base-uncased/resolve/main/special_tokens_map.json from cache at None
loading file https://huggingface.co/distilbert-base-uncased/resolve/main/tokenizer_config.json from cache at /root/.cache/huggingface/transformers/8c8624b8ac8aa99c60c912161f8332de003484428c47906d7ff7eb7f73eecdbb.20430bd8e10ef77a7d2977accef

Step,Training Loss
10,0.697600
20,0.678200
30,0.631700
40,0.702700
50,0.662600
60,0.672300
70,0.661900
80,0.680500
90,0.650900
100,0.648500




Training completed. Do not forget to share your model on huggingface.co/models =)


***** Running Prediction *****
  Num examples = 138
  Batch size = 64


PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
loading configuration file https://huggingface.co/distilbert-base-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/23454919702d26495337f3da04d1655c7ee010d5ec9d77bdb9e399e00302c0a1.d423bdf2f58dc8b77d5f5d18028d7ae4a72dcfd8f468e81fe979ada957a8c361
Model config DistilBertConfig {
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,

Step,Training Loss
10,0.702300
20,0.677100
30,0.661300
40,0.677300
50,0.661400
60,0.653600
70,0.639400
80,0.646000
90,0.651100
100,0.660200


...

Step,Training Loss
10,2.669300
20,2.560200
30,2.232100
40,1.956500
50,1.644000
60,1.585700
70,1.757400
80,1.454500
90,1.714200
100,1.578300




Training completed. Do not forget to share your model on huggingface.co/models =)


***** Running Prediction *****
  Num examples = 138
  Batch size = 64


PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
loading configuration file https://huggingface.co/distilbert-base-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/23454919702d26495337f3da04d1655c7ee010d5ec9d77bdb9e399e00302c0a1.d423bdf2f58dc8b77d5f5d18028d7ae4a72dcfd8f468e81fe979ada957a8c361
Model config DistilBertConfig {
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,

Step,Training Loss
10,2.676500
20,2.528300
30,2.223800
40,1.889000
50,1.726700
60,1.701200
70,1.645800
80,1.601000
90,1.672500
100,1.578700




Training completed. Do not forget to share your model on huggingface.co/models =)


***** Running Prediction *****
  Num examples = 137
  Batch size = 64


In [ ]:
print('PERFORMANCE')
print(classification_report(training_data['PERFORMANCE'], training_data['BERT_PERFORMANCE_Predictions'], zero_division = 1))
print('LEVEL 1')
print(classification_report(training_data['Label_num_level_1'], training_data['BERT_Label_num_level_1_Predictions'], zero_division = 1))
print('LEVEL 2')
print(classification_report(training_data['Label_num_level_2'], training_data['BERT_Label_num_level_2_Predictions'], zero_division = 1))

PERFORMANCE
              precision    recall  f1-score   support

           0       0.62      0.99      0.76       421
           1       0.73      0.06      0.11       268

    accuracy                           0.63       689
   macro avg       0.67      0.52      0.44       689
weighted avg       0.66      0.63      0.51       689

LEVEL 1
              precision    recall  f1-score   support

           0       1.00      0.00      0.00        43
           1       1.00      0.00      0.00        55
           2       1.00      0.00      0.00        87
           3       0.61      1.00      0.76       421
           4       1.00      0.00      0.00        83

    accuracy                           0.61       689
   macro avg       0.92      0.20      0.15       689
weighted avg       0.76      0.61      0.46       689

LEVEL 2
              precision    recall  f1-score   support

           0       1.00      0.00      0.00         4
           1       1.00      0.00      0.00    

In [ ]:
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')
label_count = {'PERFORMANCE':2,'Label_num_level_1':5,'Label_num_level_2':16}
k=5
seed = 3
kf = StratifiedKFold(n_splits=k,random_state=seed, shuffle=True)
for label, count in label_count.items():
  x=training_data['Concept']
  y=training_data[label]
  metrics=[]
  i=0
  for train_index, val_index in kf.split(x,y):
    train_texts, val_texts =x.iloc[train_index].tolist(),x.iloc[val_index].tolist()
    train_labels, val_labels =y.iloc[train_index].tolist(),y.iloc[val_index].tolist()

    train_encodings = tokenizer(train_texts, truncation=True, padding=True)
    val_encodings = tokenizer(val_texts, truncation=True, padding=True)


    train_dataset = OrgPerDataset(train_encodings, train_labels)
    val_dataset = OrgPerDataset(val_encodings, val_labels)

    training_args = TrainingArguments(
        output_dir='./results',          # output directory
        num_train_epochs=3,              # total number of training epochs
        per_device_train_batch_size=16,  # batch size per device during training
        per_device_eval_batch_size=64,   # batch size for evaluation
        warmup_steps=500,                # number of warmup steps for learning rate scheduler
        weight_decay=0.01,               # strength of weight decay
        logging_dir='./logs',            # directory for storing logs
        logging_steps=10,
    )
    config = DistilBertConfig.from_pretrained('distilbert-base-uncased')
    config.num_labels = count
    model = DistilBertForSequenceClassification(config)

    trainer = Trainer(
        model=model,                         # the instantiated 🤗 Transformers model to be trained
        args=training_args,                  # training arguments, defined above
        train_dataset=train_dataset,         # training dataset
        eval_dataset=val_dataset             # evaluation dataset
    )

    trainer.train()
    pred = trainer.predict(val_dataset)
    preds = pred.predictions.argmax(-1)
    indices = val_index
    training_data.loc[indices, "BERT_{}_Predictions".format(label)]=pd.Series(preds).tolist()

loading file https://huggingface.co/distilbert-base-uncased/resolve/main/vocab.txt from cache at /root/.cache/huggingface/transformers/0e1bbfda7f63a99bb52e3915dcf10c3c92122b827d92eb2d34ce94ee79ba486c.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99
loading file https://huggingface.co/distilbert-base-uncased/resolve/main/tokenizer.json from cache at /root/.cache/huggingface/transformers/75abb59d7a06f4f640158a9bfcde005264e59e8d566781ab1415b139d2e4c603.7f2721073f19841be16f41b0a70b600ca6b880c8f3df6f3535cbc704371bdfa4
loading file https://huggingface.co/distilbert-base-uncased/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/distilbert-base-uncased/resolve/main/special_tokens_map.json from cache at None
loading file https://huggingface.co/distilbert-base-uncased/resolve/main/tokenizer_config.json from cache at /root/.cache/huggingface/transformers/8c8624b8ac8aa99c60c912161f8332de003484428c47906d7ff7eb7f73eecdbb.20430bd8e10ef77a7d2977accef

Step,Training Loss
10,0.690500
20,0.679900
30,0.687600
40,0.661100
50,0.666000
60,0.652400
70,0.669800
80,0.636400
90,0.667300
100,0.673300




Training completed. Do not forget to share your model on huggingface.co/models =)


***** Running Prediction *****
  Num examples = 138
  Batch size = 64


PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
loading configuration file https://huggingface.co/distilbert-base-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/23454919702d26495337f3da04d1655c7ee010d5ec9d77bdb9e399e00302c0a1.d423bdf2f58dc8b77d5f5d18028d7ae4a72dcfd8f468e81fe979ada957a8c361
Model config DistilBertConfig {
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,

Step,Training Loss
10,0.695400
20,0.679200
30,0.673200
40,0.644700
50,0.678700
60,0.655700
70,0.670000
80,0.633300
90,0.648300
100,0.651300


...

Step,Training Loss
10,2.690600
20,2.525800
30,2.192000
40,1.812600
50,1.778000
60,1.523200
70,1.742900
80,1.546000
90,1.677000
100,1.524200




Training completed. Do not forget to share your model on huggingface.co/models =)


***** Running Prediction *****
  Num examples = 138
  Batch size = 64


PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
loading configuration file https://huggingface.co/distilbert-base-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/23454919702d26495337f3da04d1655c7ee010d5ec9d77bdb9e399e00302c0a1.d423bdf2f58dc8b77d5f5d18028d7ae4a72dcfd8f468e81fe979ada957a8c361
Model config DistilBertConfig {
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,

Step,Training Loss
10,2.681700
20,2.456300
30,2.314400
40,1.999500
50,1.756500
60,1.630100
70,1.736100
80,1.565700
90,1.545700
100,1.762100




Training completed. Do not forget to share your model on huggingface.co/models =)


***** Running Prediction *****
  Num examples = 137
  Batch size = 64


In [ ]:
print('PERFORMANCE')
print(classification_report(training_data['PERFORMANCE'], training_data['BERT_PERFORMANCE_Predictions'], zero_division = 1))
print('LEVEL 1')
print(classification_report(training_data['Label_num_level_1'], training_data['BERT_Label_num_level_1_Predictions'], zero_division = 1))
print('LEVEL 2')
print(classification_report(training_data['Label_num_level_2'], training_data['BERT_Label_num_level_2_Predictions'], zero_division = 1))

PERFORMANCE
              precision    recall  f1-score   support

           0       0.61      1.00      0.76       421
           1       0.50      0.01      0.01       268

    accuracy                           0.61       689
   macro avg       0.56      0.50      0.39       689
weighted avg       0.57      0.61      0.47       689

LEVEL 1
              precision    recall  f1-score   support

           0       1.00      0.00      0.00        43
           1       1.00      0.00      0.00        55
           2       1.00      0.00      0.00        87
           3       0.61      1.00      0.76       421
           4       1.00      0.00      0.00        83

    accuracy                           0.61       689
   macro avg       0.92      0.20      0.15       689
weighted avg       0.76      0.61      0.46       689

LEVEL 2
              precision    recall  f1-score   support

           0       1.00      0.00      0.00         4
           1       1.00      0.00      0.00    

In [ ]:
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')
label_count = {'PERFORMANCE':2,'Label_num_level_1':5,'Label_num_level_2':16}
k=5
seed = 4
kf = StratifiedKFold(n_splits=k,random_state=seed, shuffle=True)
for label, count in label_count.items():
  x=training_data['Concept']
  y=training_data[label]
  metrics=[]
  i=0
  for train_index, val_index in kf.split(x,y):
    train_texts, val_texts =x.iloc[train_index].tolist(),x.iloc[val_index].tolist()
    train_labels, val_labels =y.iloc[train_index].tolist(),y.iloc[val_index].tolist()

    train_encodings = tokenizer(train_texts, truncation=True, padding=True)
    val_encodings = tokenizer(val_texts, truncation=True, padding=True)


    train_dataset = OrgPerDataset(train_encodings, train_labels)
    val_dataset = OrgPerDataset(val_encodings, val_labels)

    training_args = TrainingArguments(
        output_dir='./results',          # output directory
        num_train_epochs=3,              # total number of training epochs
        per_device_train_batch_size=16,  # batch size per device during training
        per_device_eval_batch_size=64,   # batch size for evaluation
        warmup_steps=500,                # number of warmup steps for learning rate scheduler
        weight_decay=0.01,               # strength of weight decay
        logging_dir='./logs',            # directory for storing logs
        logging_steps=10,
    )
    config = DistilBertConfig.from_pretrained('distilbert-base-uncased')
    config.num_labels = count
    model = DistilBertForSequenceClassification(config)

    trainer = Trainer(
        model=model,                         # the instantiated 🤗 Transformers model to be trained
        args=training_args,                  # training arguments, defined above
        train_dataset=train_dataset,         # training dataset
        eval_dataset=val_dataset             # evaluation dataset
    )

    trainer.train()
    pred = trainer.predict(val_dataset)
    preds = pred.predictions.argmax(-1)
    indices = val_index
    training_data.loc[indices, "BERT_{}_Predictions".format(label)]=pd.Series(preds).tolist()

loading file https://huggingface.co/distilbert-base-uncased/resolve/main/vocab.txt from cache at /root/.cache/huggingface/transformers/0e1bbfda7f63a99bb52e3915dcf10c3c92122b827d92eb2d34ce94ee79ba486c.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99
loading file https://huggingface.co/distilbert-base-uncased/resolve/main/tokenizer.json from cache at /root/.cache/huggingface/transformers/75abb59d7a06f4f640158a9bfcde005264e59e8d566781ab1415b139d2e4c603.7f2721073f19841be16f41b0a70b600ca6b880c8f3df6f3535cbc704371bdfa4
loading file https://huggingface.co/distilbert-base-uncased/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/distilbert-base-uncased/resolve/main/special_tokens_map.json from cache at None
loading file https://huggingface.co/distilbert-base-uncased/resolve/main/tokenizer_config.json from cache at /root/.cache/huggingface/transformers/8c8624b8ac8aa99c60c912161f8332de003484428c47906d7ff7eb7f73eecdbb.20430bd8e10ef77a7d2977accef

Step,Training Loss
10,0.710400
20,0.696500
30,0.694900
40,0.693300
50,0.677900
60,0.645300
70,0.660000
80,0.639300
90,0.676800
100,0.667600




Training completed. Do not forget to share your model on huggingface.co/models =)


***** Running Prediction *****
  Num examples = 138
  Batch size = 64


PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
loading configuration file https://huggingface.co/distilbert-base-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/23454919702d26495337f3da04d1655c7ee010d5ec9d77bdb9e399e00302c0a1.d423bdf2f58dc8b77d5f5d18028d7ae4a72dcfd8f468e81fe979ada957a8c361
Model config DistilBertConfig {
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,

Step,Training Loss
10,0.687700
20,0.697000
30,0.701900
40,0.678600
50,0.644400
60,0.655600
70,0.662700
80,0.669300
90,0.672100
100,0.623400


...

Step,Training Loss
10,2.688100
20,2.565500
30,2.241800
40,1.894500
50,1.736200
60,1.625000
70,1.595800
80,1.679000
90,1.631200
100,1.523200




Training completed. Do not forget to share your model on huggingface.co/models =)


***** Running Prediction *****
  Num examples = 138
  Batch size = 64


PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
loading configuration file https://huggingface.co/distilbert-base-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/23454919702d26495337f3da04d1655c7ee010d5ec9d77bdb9e399e00302c0a1.d423bdf2f58dc8b77d5f5d18028d7ae4a72dcfd8f468e81fe979ada957a8c361
Model config DistilBertConfig {
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,

Step,Training Loss
10,2.715400
20,2.528300
30,2.186300
40,1.833600
50,1.689800
60,1.748600
70,1.716300
80,1.620700
90,1.648200
100,1.649300




Training completed. Do not forget to share your model on huggingface.co/models =)


***** Running Prediction *****
  Num examples = 137
  Batch size = 64


In [ ]:
print('PERFORMANCE')
print(classification_report(training_data['PERFORMANCE'], training_data['BERT_PERFORMANCE_Predictions'], zero_division = 1))
print('LEVEL 1')
print(classification_report(training_data['Label_num_level_1'], training_data['BERT_Label_num_level_1_Predictions'], zero_division = 1))
print('LEVEL 2')
print(classification_report(training_data['Label_num_level_2'], training_data['BERT_Label_num_level_2_Predictions'], zero_division = 1))

PERFORMANCE
              precision    recall  f1-score   support

           0       0.61      0.99      0.76       421
           1       0.40      0.01      0.01       268

    accuracy                           0.61       689
   macro avg       0.51      0.50      0.39       689
weighted avg       0.53      0.61      0.47       689

LEVEL 1
              precision    recall  f1-score   support

           0       1.00      0.00      0.00        43
           1       1.00      0.00      0.00        55
           2       1.00      0.00      0.00        87
           3       0.61      1.00      0.76       421
           4       1.00      0.00      0.00        83

    accuracy                           0.61       689
   macro avg       0.92      0.20      0.15       689
weighted avg       0.76      0.61      0.46       689

LEVEL 2
              precision    recall  f1-score   support

           0       1.00      0.00      0.00         4
           1       1.00      0.00      0.00    

In [ ]:
!pip install detext==2.0.8

     |████████████████████████████████| 66 kB 1.9 MB/s 
     |████████████████████████████████| 17.3 MB 67 kB/s 
     |████████████████████████████████| 109.3 MB 38 kB/s 
     |████████████████████████████████| 40 kB 3.9 MB/s 
     |████████████████████████████████| 50 kB 3.9 MB/s 
     |████████████████████████████████| 488 kB 47.0 MB/s 
     |████████████████████████████████| 3.1 MB 26.0 MB/s 
  Created wheel for detext: filename=detext-2.0.8-py3-none-any.whl size=77058 sha256=172a72bd78105f43233ceca7e239a55ffd907a56ccdcde07b6a530f94ab9f871
  Stored in directory: /root/.cache/pip/wheels/28/7c/f2/307486854269edf085f24481171405cea10a2f7ca2e55ae7d6
  Created wheel for gast: filename=gast-0.2.2-py3-none-any.whl size=7553 sha256=efcb80504d15ae77832ac03cd64128c015f150cb28eac5ae769664eea9fc8190
  Stored in directory: /root/.cache/pip/wheels/21/7f/02/420f32a803f7d0967b48dd823da3f558c5166991bfd204eef3
Successfully built detext gast
  Attempting uninstall: numpy
    Found existing installation

In [ ]:
%tensorflow_version 1.x
import tensorflow as tf
tf.__version__

TensorFlow 1.x selected.


'1.15.2'

In [ ]:
# # The constants
# CLASSES = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15]
# See https://github.com/linkedin/detext/blob/master/TRAINING.md for more details on the feature naming format required by DeText
QUERY_FIELDNAME = "doc_query"
WIDE_FTR_FIELDNAME = "wide_ftrs"
LABEL_FIELDNAME = "label"
VOCAB_FILE = "vocab.txt"

In [ ]:
from tensorflow.contrib import predictor
from detext.run_detext import run_detext,DetextArg

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.





In [ ]:
import tensorflow as tf

def create_float_feature(value):
    feature = tf.train.Feature(float_list=tf.train.FloatList(value=[value]))
    return feature


def create_byte_feature(value):
    feature = tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))
    return feature


def create_tfrecords(X, y, output_file):
  assert len(X) == len(y)
  total_records = 0
  writer = tf.python_io.TFRecordWriter(output_file)
  for q, c in zip(X, y):
    features = {}
    features[QUERY_FIELDNAME] = create_byte_feature(str.encode(q))
    features[LABEL_FIELDNAME] = create_float_feature(c)
    features[WIDE_FTR_FIELDNAME] = create_float_feature(len(q.split()))
    tf_example = tf.train.Example(features=tf.train.Features(feature=features))
    if total_records < 2:
      print(tf_example)
    writer.write(tf_example.SerializeToString())
    total_records += 1
  print("processed {} records".format(total_records))

In [ ]:
def predict_class(query):
  predictions = predict_fn(
                  {"doc_query": [query],
                   "wide_ftrs": [[len(query.split())]],
                   # dummy pass-through input for label field -- this is not used in making predictions
                   "label": [1.0]})
  probabilities = predictions['multiclass_probabilities'][0]
  predicted_class = np.argmax(probabilities)
  # print("query \"{}\", predicted intent \"{}\".".format(query, predicted_class))
  return predicted_class

In [ ]:
import glob, os
import numpy as np

In [ ]:
#K-Fold cross validation with seed=1 and k=5
k=5
seed = 1
kf = StratifiedKFold(n_splits=k,random_state=seed, shuffle=True)
label_count = {'PERFORMANCE':2,'Label_num_level_1':5,'Label_num_level_2':16}
for label, count in label_count.items():
  x=training_data
  y=training_data[label]
  metrics=[]
  i=0
  for train_index, val_index in kf.split(x,y):
    train_texts, val_texts =x['Concept'].iloc[train_index],x['Concept'].iloc[val_index]
    train_labels, val_labels =y.iloc[train_index],y.iloc[val_index]
    X_train, X_dev, y_train, y_dev = train_test_split(train_texts, train_labels, test_size=0.1, random_state=42)
    create_tfrecords(X_train, y_train, 'train.tfrecord')
    create_tfrecords(X_dev, y_dev, 'dev.tfrecord')
    create_tfrecords(val_texts, val_labels, 'test.tfrecord')
    count_vect = CountVectorizer()
    X = count_vect.fit_transform(X_train)
    vocab = ['[PAD]', '[UNK]']
    vocab.extend(list(count_vect.vocabulary_.keys()))
    # Write vocab to file
    with open(VOCAB_FILE, 'w') as f:
      for v in vocab:
        f.write("{}\n".format(v))

    args = DetextArg(num_classes=count,
                ftr_ext="cnn",
                num_filters=50,
                num_units=64,
                num_wide=1,
                optimizer="bert_adam", # same AdamWeightDecay optimizer as in BERT training
                learning_rate=0.001,
                max_len=16,
                min_len=3,
                use_deep=True,
                num_train_steps=300,
                steps_per_stats=1,
                steps_per_eval=30,
                train_batch_size=64,
                test_batch_size=64,
                pmetric="accuracy",
                all_metrics=["accuracy", "confusion_matrix"],
                vocab_file=VOCAB_FILE,
                feature_names=["label","doc_query","wide_ftrs"],
                train_file="train.tfrecord",
                dev_file="dev.tfrecord",
                test_file="test.tfrecord",
                out_dir="output")
    run_detext(args)
    saved_model_path = glob.glob('output/export/best_accuracy/*')[0]
    predict_fn = predictor.from_saved_model(saved_model_path)
    list_test = val_texts.to_list()
    classification_prediction_list = []
    for c in list_test:
      classification_prediction_list.append(predict_class(c))
    # Naive_Bayes Model metrics
    classification_list = val_labels.to_list()
    indices = val_index
    training_data.loc[indices, "DeText_{}_Predictions".format(label)]=classification_prediction_list

Streaming output truncated to the last 5000 lines.
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  2  2  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  2  0  0  0  0]]
INFO:tensorflow:Summary for np.ndarray is not visible in Tensorboard by default. Consider using a Tensorboard plugin for visualization (see https://github.com/tensorflow/tensorboard-plugin-example/blob/master/README.md for more information).
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 240: output/model.ckpt-240
INFO:tensorflow:## Step 240
INFO:tensorflow:loss : 1.4923008680343628
INFO:tensorflow:[[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  1  0  0  0  0]
 [ 0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  3  0  0  0  0  0  0  1  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  

In [ ]:
print("DeText")
print("PERFORMANCE")
print(classification_report(training_data['PERFORMANCE'], training_data['DeText_PERFORMANCE_Predictions'], zero_division = 1))
print("Label_num_level_1")
print(classification_report(training_data['Label_num_level_1'], training_data['DeText_Label_num_level_1_Predictions'], zero_division = 1))
print("Label_num_level_2")
print(classification_report(training_data['Label_num_level_2'], training_data['DeText_Label_num_level_2_Predictions'], zero_division = 1))

INFO:tensorflow:DeText
INFO:tensorflow:PERFORMANCE
INFO:tensorflow:              precision    recall  f1-score   support

           0       0.83      0.83      0.83       421
           1       0.73      0.73      0.73       268

    accuracy                           0.79       689
   macro avg       0.78      0.78      0.78       689
weighted avg       0.79      0.79      0.79       689

INFO:tensorflow:Label_num_level_1
INFO:tensorflow:              precision    recall  f1-score   support

           0       0.61      0.47      0.53        43
           1       0.68      0.58      0.63        55
           2       0.74      0.67      0.70        87
           3       0.82      0.87      0.85       421
           4       0.69      0.71      0.70        83

    accuracy                           0.78       689
   macro avg       0.71      0.66      0.68       689
weighted avg       0.77      0.78      0.77       689

INFO:tensorflow:Label_num_level_2
INFO:tensorflow:              pre

In [ ]:
#K-Fold cross validation with seed=1 and k=5
k=5
seed = 2
kf = StratifiedKFold(n_splits=k,random_state=seed, shuffle=True)
label_count = {'PERFORMANCE':2,'Label_num_level_1':5,'Label_num_level_2':16}
for label, count in label_count.items():
  x=training_data
  y=training_data[label]
  metrics=[]
  i=0
  for train_index, val_index in kf.split(x,y):
    train_texts, val_texts =x['Concept'].iloc[train_index],x['Concept'].iloc[val_index]
    train_labels, val_labels =y.iloc[train_index],y.iloc[val_index]
    X_train, X_dev, y_train, y_dev = train_test_split(train_texts, train_labels, test_size=0.1, random_state=42)
    create_tfrecords(X_train, y_train, 'train.tfrecord')
    create_tfrecords(X_dev, y_dev, 'dev.tfrecord')
    create_tfrecords(val_texts, val_labels, 'test.tfrecord')
    count_vect = CountVectorizer()
    X = count_vect.fit_transform(X_train)
    vocab = ['[PAD]', '[UNK]']
    vocab.extend(list(count_vect.vocabulary_.keys()))
    # Write vocab to file
    with open(VOCAB_FILE, 'w') as f:
      for v in vocab:
        f.write("{}\n".format(v))

    args = DetextArg(num_classes=count,
                ftr_ext="cnn",
                num_filters=50,
                num_units=64,
                num_wide=1,
                optimizer="bert_adam", # same AdamWeightDecay optimizer as in BERT training
                learning_rate=0.001,
                max_len=16,
                min_len=3,
                use_deep=True,
                num_train_steps=300,
                steps_per_stats=1,
                steps_per_eval=30,
                train_batch_size=64,
                test_batch_size=64,
                pmetric="accuracy",
                all_metrics=["accuracy", "confusion_matrix"],
                vocab_file=VOCAB_FILE,
                feature_names=["label","doc_query","wide_ftrs"],
                train_file="train.tfrecord",
                dev_file="dev.tfrecord",
                test_file="test.tfrecord",
                out_dir="output")
    run_detext(args)
    saved_model_path = glob.glob('output/export/best_accuracy/*')[0]
    predict_fn = predictor.from_saved_model(saved_model_path)
    list_test = val_texts.to_list()
    classification_prediction_list = []
    for c in list_test:
      classification_prediction_list.append(predict_class(c))
    # Naive_Bayes Model metrics
    classification_list = val_labels.to_list()
    indices = val_index
    training_data.loc[indices, "DeText_{}_Predictions".format(label)]=classification_prediction_list

Streaming output truncated to the last 5000 lines.
 [ 0  0  0  0  0  0  0  0  0  0  0  0  1  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  3  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0]]
INFO:tensorflow:Checking checkpoint model.ckpt-240
INFO:tensorflow:skipping checkpoint model.ckpt-240 with metric/accuracy = 0.8928571343421936
INFO:tensorflow:global_step/sec: 0.76322
INFO:tensorflow:loss = 0.18010569, step = 239 (1.301 sec)
INFO:tensorflow:global_step/sec: 31.1425
INFO:tensorflow:loss = 0.069924645, step = 240 (0.033 sec)
INFO:tensorflow:global_step = 240, loss = 0.069924645 (1.680 sec)
INFO:tensorflow:global_step/sec: 29.468
INFO:tensorflow:loss = 0.07822089, step = 241 (0.035 sec)
INFO:tensorflow:global_step/sec: 12.8007
INFO:tensorflow:loss = 0.07059306, step = 242 (0.081 sec)
INFO:tensorflow:global_step/sec: 23.3109
INFO:tensorflow:loss = 0.14557658, step = 243 (0.043 sec)
INFO:tensorflow:global_step/sec: 14.416

In [ ]:
print("DeText")
print("PERFORMANCE")
print(classification_report(training_data['PERFORMANCE'], training_data['DeText_PERFORMANCE_Predictions'], zero_division = 1))
print("Label_num_level_1")
print(classification_report(training_data['Label_num_level_1'], training_data['DeText_Label_num_level_1_Predictions'], zero_division = 1))
print("Label_num_level_2")
print(classification_report(training_data['Label_num_level_2'], training_data['DeText_Label_num_level_2_Predictions'], zero_division = 1))

INFO:tensorflow:DeText
INFO:tensorflow:PERFORMANCE
INFO:tensorflow:              precision    recall  f1-score   support

           0       0.84      0.86      0.85       421
           1       0.77      0.74      0.75       268

    accuracy                           0.81       689
   macro avg       0.80      0.80      0.80       689
weighted avg       0.81      0.81      0.81       689

INFO:tensorflow:Label_num_level_1
INFO:tensorflow:              precision    recall  f1-score   support

           0       0.54      0.35      0.42        43
           1       0.71      0.58      0.64        55
           2       0.65      0.67      0.66        87
           3       0.82      0.87      0.84       421
           4       0.73      0.71      0.72        83

    accuracy                           0.77       689
   macro avg       0.69      0.64      0.66       689
weighted avg       0.76      0.77      0.76       689

INFO:tensorflow:Label_num_level_2
INFO:tensorflow:              pre

In [ ]:
#K-Fold cross validation with seed=1 and k=5
k=5
seed = 3
kf = StratifiedKFold(n_splits=k,random_state=seed, shuffle=True)
label_count = {'PERFORMANCE':2,'Label_num_level_1':5,'Label_num_level_2':16}
for label, count in label_count.items():
  x=training_data
  y=training_data[label]
  metrics=[]
  i=0
  for train_index, val_index in kf.split(x,y):
    train_texts, val_texts =x['Concept'].iloc[train_index],x['Concept'].iloc[val_index]
    train_labels, val_labels =y.iloc[train_index],y.iloc[val_index]
    X_train, X_dev, y_train, y_dev = train_test_split(train_texts, train_labels, test_size=0.1, random_state=42)
    create_tfrecords(X_train, y_train, 'train.tfrecord')
    create_tfrecords(X_dev, y_dev, 'dev.tfrecord')
    create_tfrecords(val_texts, val_labels, 'test.tfrecord')
    count_vect = CountVectorizer()
    X = count_vect.fit_transform(X_train)
    vocab = ['[PAD]', '[UNK]']
    vocab.extend(list(count_vect.vocabulary_.keys()))
    # Write vocab to file
    with open(VOCAB_FILE, 'w') as f:
      for v in vocab:
        f.write("{}\n".format(v))

    args = DetextArg(num_classes=count,
                ftr_ext="cnn",
                num_filters=50,
                num_units=64,
                num_wide=1,
                optimizer="bert_adam", # same AdamWeightDecay optimizer as in BERT training
                learning_rate=0.001,
                max_len=16,
                min_len=3,
                use_deep=True,
                num_train_steps=300,
                steps_per_stats=1,
                steps_per_eval=30,
                train_batch_size=64,
                test_batch_size=64,
                pmetric="accuracy",
                all_metrics=["accuracy", "confusion_matrix"],
                vocab_file=VOCAB_FILE,
                feature_names=["label","doc_query","wide_ftrs"],
                train_file="train.tfrecord",
                dev_file="dev.tfrecord",
                test_file="test.tfrecord",
                out_dir="output")
    run_detext(args)
    saved_model_path = glob.glob('output/export/best_accuracy/*')[0]
    predict_fn = predictor.from_saved_model(saved_model_path)
    list_test = val_texts.to_list()
    classification_prediction_list = []
    for c in list_test:
      classification_prediction_list.append(predict_class(c))
    # Naive_Bayes Model metrics
    classification_list = val_labels.to_list()
    indices = val_index
    training_data.loc[indices, "DeText_{}_Predictions".format(label)]=classification_prediction_list

Streaming output truncated to the last 5000 lines.
 [ 0  0  0  0  0  0  0  0  0  0  0  1  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  1  2  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  1  1  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1]]
INFO:tensorflow:Summary for np.ndarray is not visible in Tensorboard by default. Consider using a Tensorboard plugin for visualization (see https://github.com/tensorflow/tensorboard-plugin-example/blob/master/README.md for more information).
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 240: output/model.ckpt-240
INFO:tensorflow:## Step 240
INFO:tensorflow:loss : 1.6876583099365234
INFO:tensorflow:[[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  2  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  2  0  0  0  0  0  0  2  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  

In [ ]:
print("DeText")
print("PERFORMANCE")
print(classification_report(training_data['PERFORMANCE'], training_data['DeText_PERFORMANCE_Predictions'], zero_division = 1))
print("Label_num_level_1")
print(classification_report(training_data['Label_num_level_1'], training_data['DeText_Label_num_level_1_Predictions'], zero_division = 1))
print("Label_num_level_2")
print(classification_report(training_data['Label_num_level_2'], training_data['DeText_Label_num_level_2_Predictions'], zero_division = 1))

INFO:tensorflow:DeText
INFO:tensorflow:PERFORMANCE
INFO:tensorflow:              precision    recall  f1-score   support

           0       0.84      0.80      0.82       421
           1       0.71      0.76      0.74       268

    accuracy                           0.79       689
   macro avg       0.78      0.78      0.78       689
weighted avg       0.79      0.79      0.79       689

INFO:tensorflow:Label_num_level_1
INFO:tensorflow:              precision    recall  f1-score   support

           0       0.61      0.44      0.51        43
           1       0.69      0.56      0.62        55
           2       0.72      0.67      0.69        87
           3       0.81      0.87      0.84       421
           4       0.71      0.69      0.70        83

    accuracy                           0.77       689
   macro avg       0.71      0.65      0.67       689
weighted avg       0.77      0.77      0.77       689

INFO:tensorflow:Label_num_level_2
INFO:tensorflow:              pre

In [ ]:
#K-Fold cross validation with seed=1 and k=5
k=5
seed = 4
kf = StratifiedKFold(n_splits=k,random_state=seed, shuffle=True)
label_count = {'PERFORMANCE':2,'Label_num_level_1':5,'Label_num_level_2':16}
for label, count in label_count.items():
  x=training_data
  y=training_data[label]
  metrics=[]
  i=0
  for train_index, val_index in kf.split(x,y):
    train_texts, val_texts =x['Concept'].iloc[train_index],x['Concept'].iloc[val_index]
    train_labels, val_labels =y.iloc[train_index],y.iloc[val_index]
    X_train, X_dev, y_train, y_dev = train_test_split(train_texts, train_labels, test_size=0.1, random_state=42)
    create_tfrecords(X_train, y_train, 'train.tfrecord')
    create_tfrecords(X_dev, y_dev, 'dev.tfrecord')
    create_tfrecords(val_texts, val_labels, 'test.tfrecord')
    count_vect = CountVectorizer()
    X = count_vect.fit_transform(X_train)
    vocab = ['[PAD]', '[UNK]']
    vocab.extend(list(count_vect.vocabulary_.keys()))
    # Write vocab to file
    with open(VOCAB_FILE, 'w') as f:
      for v in vocab:
        f.write("{}\n".format(v))

    args = DetextArg(num_classes=count,
                ftr_ext="cnn",
                num_filters=50,
                num_units=64,
                num_wide=1,
                optimizer="bert_adam", # same AdamWeightDecay optimizer as in BERT training
                learning_rate=0.001,
                max_len=16,
                min_len=3,
                use_deep=True,
                num_train_steps=300,
                steps_per_stats=1,
                steps_per_eval=30,
                train_batch_size=64,
                test_batch_size=64,
                pmetric="accuracy",
                all_metrics=["accuracy", "confusion_matrix"],
                vocab_file=VOCAB_FILE,
                feature_names=["label","doc_query","wide_ftrs"],
                train_file="train.tfrecord",
                dev_file="dev.tfrecord",
                test_file="test.tfrecord",
                out_dir="output")
    run_detext(args)
    saved_model_path = glob.glob('output/export/best_accuracy/*')[0]
    predict_fn = predictor.from_saved_model(saved_model_path)
    list_test = val_texts.to_list()
    classification_prediction_list = []
    for c in list_test:
      classification_prediction_list.append(predict_class(c))
    # Naive_Bayes Model metrics
    classification_list = val_labels.to_list()
    indices = val_index
    training_data.loc[indices, "DeText_{}_Predictions".format(label)]=classification_prediction_list

Streaming output truncated to the last 5000 lines.
 [ 0  0  0  0  2  0  0  0  0  0  0  0  0  0  0  1]
 [ 0  0  0  0  0  0  0  0  0  0  0  1  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  1  0  0  0  0]
 [ 0  0  0  0  0  0  0  5  0  2  0  1  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  1  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  1  1  0  0  0 28  0  1  0  2]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  1  1  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  1  0  1  1  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1]] : 
INFO:tensorflow:[[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  1  0  0  0  1  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  2  0  0  0  0  0  0  0  0  0  0  1]
 [ 0  0  0  0  0  0  0  0  0  0  0  1  0  0  0  0]
 [ 0  0  0 

In [ ]:
print("DeText")
print("PERFORMANCE")
print(classification_report(training_data['PERFORMANCE'], training_data['DeText_PERFORMANCE_Predictions'], zero_division = 1))
print("Label_num_level_1")
print(classification_report(training_data['Label_num_level_1'], training_data['DeText_Label_num_level_1_Predictions'], zero_division = 1))
print("Label_num_level_2")
print(classification_report(training_data['Label_num_level_2'], training_data['DeText_Label_num_level_2_Predictions'], zero_division = 1))

INFO:tensorflow:DeText
INFO:tensorflow:PERFORMANCE
INFO:tensorflow:              precision    recall  f1-score   support

           0       0.83      0.83      0.83       421
           1       0.74      0.74      0.74       268

    accuracy                           0.80       689
   macro avg       0.78      0.79      0.78       689
weighted avg       0.80      0.80      0.80       689

INFO:tensorflow:Label_num_level_1
INFO:tensorflow:              precision    recall  f1-score   support

           0       0.53      0.47      0.49        43
           1       0.74      0.45      0.56        55
           2       0.72      0.67      0.69        87
           3       0.80      0.88      0.84       421
           4       0.74      0.69      0.71        83

    accuracy                           0.77       689
   macro avg       0.71      0.63      0.66       689
weighted avg       0.76      0.77      0.76       689

INFO:tensorflow:Label_num_level_2
INFO:tensorflow:              pre